In [23]:
def select_region(system, feature, pcg, what, group_data, group_metadata=[]):
    # plot synthetic dataset only # used to be 'friends' != True here... but no point in it. move on with sth basic
    select_criterion = (group_data['Arch'] != '')
    if(system!=''):
        select_criterion = select_criterion & (group_data['System']==system)
    select_str = ''

    # features = ['A_mem_footprint','avg_nz_row', 'skew_coeff', 'avg_num_neighbours', 'cross_row_similarity']
    features = ['A_mem_footprint','avg_nz_row', 'skew_coeff', 'regularity']
    for feat in features:
        if(feat=='regularity'):
            if(feat!=feature):
                irr_val_list = ['SS', 'SM', 'SL', 'MS', 'MM', 'ML', 'LS', 'LM', 'LL']

                select_str = select_str + feat + '=['+ ','.join(irr_val_list) +']' + '  '
                for irr_val in irr_val_list:
                    if(irr_val == irr_val_list[0]):
                        select_criterion_irr = (group_data[feat] == irr_val)
                    else:
                        select_criterion_irr = select_criterion_irr | (group_data[feat] == irr_val)
                select_criterion = select_criterion & select_criterion_irr
        else:
            if(group_metadata!=[]):
                feat_range = list(group_metadata[feat])[0].strip('[ ]').split(' - ')
                min_feat, max_feat = float(feat_range[0]), float(feat_range[1])
            else:
                if(feat!=feature):
                    if(feat == 'A_mem_footprint'):
                        min_feat, max_feat = 0, 2048
                    if(feat == 'avg_nz_row'):
                        min_feat, max_feat = 0, 510
                    if(feat == 'skew_coeff'):
                        prmin_feat, max_feat = 0, 180000
                    select_str = select_str + feat + '=['+str(min_feat)+'-'+str(max_feat)+']' + ' '
                    select_criterion = select_criterion & (group_data[feat] >= min_feat) & (group_data[feat] <= max_feat)
    return select_criterion, select_str


# plot impact of each feature together for all platforms

In [24]:
font = 15
def plot_dev_perf(feature, pcg, what, group_data, group_metadata=[]):
    system2 = 'Feature Impact'
    plot_x_axis = feature
    if(feature=='A_mem_footprint'):
        plot_x_axis = 'mem_range'
    elif(feature=='avg_nz_row'):
        plot_x_axis = 'anr_categ'
    elif(feature=='skew_coeff'):
        plot_x_axis = 'skew_categ'

    plot_y_axis = 'gflops'

    plot_id = system2
    select_str = feature

    ctr = 0
    fig, axs = plt.subplots(len(ranges_dev))
    fig.subplots_adjust(left=.15, bottom=.17*(2/6)*(4/3), right=.99, top=.99)
    for system in ranges_dev:
        select_criterion_tmp, select_str_tmp = select_region(system, feature, pcg, what, group_data, group_metadata=[])
        group_system_select_tmp = group_data[select_criterion_tmp]
        
        group_system_select_tmp_sorted = group_system_select_tmp
        if plot_x_axis == 'mem_range':
            cat_it = CategoricalDtype(ranges_memr, ordered=True)
        if plot_x_axis == 'regularity':
            cat_it = CategoricalDtype(ranges_regularity, ordered=True)
        if plot_x_axis == 'anr_categ':
            cat_it = CategoricalDtype(ranges_anr, ordered=True)
        if plot_x_axis == 'skew_categ':
            cat_it = CategoricalDtype(ranges_skew, ordered=True)

        pd.options.mode.chained_assignment = None # https://stackoverflow.com/a/49729413
        group_system_select_tmp_sorted[plot_x_axis] = group_system_select_tmp_sorted[plot_x_axis].astype(cat_it)
        group_system_select_tmp_sorted = group_system_select_tmp_sorted.sort_values(plot_x_axis)
        
        sns_plot = sns.boxplot(fliersize = 0.3, linewidth=0.6, width=0.4, data=group_system_select_tmp, x=plot_x_axis, y=plot_y_axis, 
                               hue='System', hue_order=ranges_dev, ax=axs[ctr], dodge=False)

        axs[ctr].tick_params(axis='both', which='major', labelsize=font)
        axs[ctr].tick_params(axis='both', which='minor', labelsize=font)
        axs[ctr].set_ylabel('')

        if(plot_y_axis == 'gflops'):
            axs[ctr].set_ylim(y_limit_dictionary[system])

        if(system == ranges_dev[-1]):  # only for last platform
            if(plot_x_axis == 'mem_range'):
                axs[ctr].set_xlabel('memory range (MB)', fontsize = font)
            if(plot_x_axis == 'anr_categ'):
                axs[ctr].set_xlabel('average number of nonzeros per row', fontsize = font)
            if(plot_x_axis == 'skew_categ'):
                axs[ctr].set_xlabel('skew coefficient', fontsize = font)
            if(plot_x_axis == 'regularity'):
                axs[ctr].set_xlabel('regularity ( cross_row_similarity / avg_num_neighbours )', fontsize = font)
        else:
            axs[ctr].set_xlabel('')

        if(system != ranges_dev[-1]): # for all except last platform
            axs[ctr].set_xticks([])
            axs[ctr].set_xticklabels([])

        # axs[ctr].text( 0.02, 0.9, system, fontsize=font, transform = axs[ctr].transAxes, )

        plt.xticks(rotation=90)#rotation
        # no_legend = True
        # if(no_legend==True):
        #     axs[ctr].get_legend().remove()

        # whichever system is first in ranges_dev
        if(system == ranges_dev[0]):
            axs[ctr].legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=len(ranges_dev), frameon=False)
            handles, labels = axs[ctr].get_legend_handles_labels()
        else:
            axs[ctr].get_legend().remove()

        plt.xticks(rotation=45)#rotation)
        ctr+=1

    select_str_id = select_str.replace('=', 'eq').replace('<', 'l').replace('>', 'g').replace(', ', '_').replace(',', '_').replace('\n', '_').replace(' ', '-')
    if('-performance_' in select_str_id):
        select_str_id = select_str_id.split('-performance_')[0]
    else:
        select_str_id = select_str_id.split('__')[0]

    # fig.set_size_inches(width*1.2, height*1.7)
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)
    fig.text(0.00, 0.5, 'GFLOPs', va='center', rotation='vertical', fontsize=font)
    fig_name = './Plots/%s_Select-%s_x-%s_y-%s.pdf' % (plot_id, select_str_id, plot_x_axis, plot_y_axis)
    fig.savefig(fig_name)
    plt.close()
    subprocess.run(['pdfcrop', fig_name], stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
    subprocess.run(['mv', fig_name.replace('.pdf', '-crop.pdf'), fig_name])

    return handles, labels # to be used in later plots

#####################################################################################################################
#####################################################################################################################

group_data = group_system_best

features = ['A_mem_footprint', 'avg_nz_row', 'skew_coeff', 'regularity']

ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'Grace-H100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
plt.rc('figure', figsize=(4*width,len(ranges_dev)*0.5*height))

change_colors(ranges_dev, color_dictionary)

handles, labels = [], []
for feature in features:
    # if(system=='Tesla-V100'): # if(system=='AMD-EPYC'): # if(system=='ARM-NEON'): # if(system=='Alveo-U280'):
    handles, labels = plot_dev_perf(feature, 10, 'all', group_data)


---
# Plot histograms of performance on each device separately.
# View range of performance for each format tested

In [25]:
def plot_histogram_performance_device(data, which, System):
    data_sys = data[data['System'] == System]
    sns.histplot(data=data_sys, x='gflops', hue='implementation', element='step')
    plt.title('Histogram of performance (per format) (%s) for %s' % (which, System))
    plt.tight_layout()
    fig_name = './Plots/histogram/histogram-%s-%s.pdf' % (System, which)
    plt.savefig(fig_name)
    plt.close()
    subprocess.run(['pdfcrop', fig_name], stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
    subprocess.run(['mv', fig_name.replace('.pdf', '-crop.pdf'), fig_name])


def plot_histogram_performance_device_more_specific(data, which, System, feature, feature_values):
    data_sys = data[data['System'] == System]
    data_sys = data_sys[data_sys[feature].isin(feature_values) ]
    sns.histplot(data=data_sys, x='gflops', hue='implementation', element='step')
    plt.title('Histogram of performance (per format) (%s) for %s for %s = %s' % (which, System, feature, feature_values))
    plt.tight_layout()
    fig_name = './Plots/histogram/histogram-specific-%s-%s.pdf' % (System, which)
    plt.savefig(fig_name)
    plt.close()
    subprocess.run(['pdfcrop', fig_name], stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
    subprocess.run(['mv', fig_name.replace('.pdf', '-crop.pdf'), fig_name])


In [26]:
plt.rc('figure', figsize=(2*width,2*height))
    
ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'Grace-H100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
for System in ranges_dev:
    plot_histogram_performance_device(group_system_best, 'bestof', System)
    plot_histogram_performance_device(groupreps, 'all', System)

feature = 'mem_range'
feature_values = ['[256-512]', '[512-1024]', '[1024-2048]']
for System in ranges_dev:
    plot_histogram_performance_device_more_specific(group_system_best, 'bestof', System, feature, feature_values)
    plot_histogram_performance_device_more_specific(groupreps, 'all', System, feature, feature_values)


In [27]:
def plot_per_feature_below(data, thresholds, feature, bins, System, verbose=False):
    # sns.histplot(data=data, x=feature, hue='implementation', bins=bins)
    sns.histplot(data=data, x=feature, bins=bins)

    if(verbose):
        # Access the patches (bars) in the histogram
        patches = plt.gca().patches
        # Add xtick values and bar heights as labels
        for patch in patches:
            height = patch.get_height()
            if(height>0):
                x = patch.get_x() + patch.get_width() / 2
                plt.text(x, height+1, f'{x:.2f} ({height})', ha='center', va='bottom', rotation=45)

    plt.title('Variation of %s for %s GFLOPs performance on %s' % (feature, thresholds, System))
    plt.tight_layout()
    fig_name = './Plots/histogram/histogram-%s-%s-%s.pdf' % (System, thresholds, feature)
    plt.savefig(fig_name)
    plt.close()
    subprocess.run(['pdfcrop', fig_name], stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
    subprocess.run(['mv', fig_name.replace('.pdf', '-crop.pdf'), fig_name])


def filter_data_performance(data, System, features, features_values, perf_thresholds):
    print('Filter applied to data on device %s: ' % (System))
    filter_data = data
    filter_data = filter_data[filter_data['System'] == System]
    for i in range(len(features)):
        print(features[i], features_values[i])
        filter_data = filter_data[filter_data[feature].isin(feature_values)]
    print()
    print('Performance range of', System, 'for these matrices:', round(filter_data['gflops'].min(),2), '-', round(filter_data['gflops'].max(),2))
    features = ['avg_nz_row', 'skew_coeff', 'avg_num_neighbours', 'cross_row_similarity']
    bins = [20, 100, 20, 20]
    rounding = [0, -1, 1, 1]

    filter_data_thres = filter_data[(filter_data['gflops'] >= perf_thresholds[0]) & (filter_data['gflops'] <= perf_thresholds[1])]
    print(filter_data_thres.shape[0],'/', filter_data.shape[0], 'matrices achieve performance of', perf_thresholds, 'GFLOPs')  
    print('Implementations of performance in range ', perf_thresholds, set(filter_data_thres['implementation']))
    for i in range(len(features)):
        plot_per_feature_below(filter_data_thres, perf_thresholds, features[i], bins[i], System, verbose=True)
        print(perf_thresholds, '->', features[i], sorted(set([round(x,rounding[i]) for x in list(filter_data_thres[features[i]])])))
    print()


In [28]:
features = ['mem_range']
features_values = [['[1024-2048]']]
filter_data_performance(group_system_best, 'Tesla-A100', features, features_values, [100,150])

Filter applied to data on device Tesla-A100: 
mem_range ['[1024-2048]']

Performance range of Tesla-A100 for these matrices: 27.61 - 220.43
2567 / 7619 matrices achieve performance of [100, 150] GFLOPs
Implementations of performance in range  [100, 150] {'cu-CSR', 'Merge', 'SELL-C-s', 'cu-COO'}
[100, 150] -> avg_nz_row [5.0, 10.0, 20.0, 50.0, 51.0, 100.0, 101.0, 499.0, 500.0, 501.0]
[100, 150] -> skew_coeff [0.0, 100.0, 110.0, 140.0, 180.0, 280.0, 390.0, 500.0, 610.0, 1000.0, 2760.0, 3590.0, 4410.0, 7060.0, 9710.0, 9820.0, 9860.0, 9890.0, 9920.0, 9930.0, 9940.0, 9950.0, 9960.0, 9970.0, 9980.0, 9990.0, 10000.0]
[100, 150] -> avg_num_neighbours [0.0, 0.1, 0.2, 0.4, 0.5, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8]
[100, 150] -> cross_row_similarity [0.0, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 0.9, 1.0]



In [29]:
features = ['mem_range']
# features_values = [['[256-512]','[512-1024]','[1024-2048]']]
features_values = [['[1024-2048]']]
filter_data_performance(group_system_best, 'Tesla-A100', features, features_values, [0,50])

Filter applied to data on device Tesla-A100: 
mem_range ['[1024-2048]']

Performance range of Tesla-A100 for these matrices: 27.61 - 220.43
182 / 7619 matrices achieve performance of [0, 50] GFLOPs
Implementations of performance in range  [0, 50] {'cu-CSR', 'SELL-C-s'}
[0, 50] -> avg_nz_row [5.0, 10.0]
[0, 50] -> skew_coeff [0.0, 100.0, 1000.0, 9980.0, 9990.0, 10000.0]
[0, 50] -> avg_num_neighbours [0.0, 0.4, 0.5, 0.7, 0.8, 0.9, 1.1]
[0, 50] -> cross_row_similarity [0.0, 0.1, 0.4, 0.5]



In [30]:
features = ['mem_range']
# features_values = [['[256-512]','[512-1024]','[1024-2048]']]
features_values = [['[1024-2048]']]
filter_data_performance(group_system_best, 'Tesla-A100', features, features_values, [0,75])

Filter applied to data on device Tesla-A100: 
mem_range ['[1024-2048]']

Performance range of Tesla-A100 for these matrices: 27.61 - 220.43
507 / 7619 matrices achieve performance of [0, 75] GFLOPs
Implementations of performance in range  [0, 75] {'cu-CSR', 'SELL-C-s'}
[0, 75] -> avg_nz_row [5.0, 10.0, 20.0]
[0, 75] -> skew_coeff [0.0, 100.0, 1000.0, 9980.0, 9990.0, 10000.0]
[0, 75] -> avg_num_neighbours [0.0, 0.4, 0.5, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.5]
[0, 75] -> cross_row_similarity [0.0, 0.1, 0.2, 0.4, 0.5]



In [37]:
features = ['mem_range']
features_values = [['[1024-2048]']]
filter_data_performance(group_system_best, 'Grace-H100', features, features_values, [150,250])

Filter applied to data on device Grace-H100: 
mem_range ['[1024-2048]']

Performance range of Grace-H100 for these matrices: 76.49 - 605.5
384 / 7553 matrices achieve performance of [150, 250] GFLOPs
Implementations of performance in range  [150, 250] {'cu-CSR'}
[150, 250] -> avg_nz_row [5.0, 10.0, 20.0, 50.0]
[150, 250] -> skew_coeff [0.0, 100.0, 1000.0, 9980.0, 9990.0, 10000.0]
[150, 250] -> avg_num_neighbours [0.0, 0.1, 0.2, 0.4, 0.5, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
[150, 250] -> cross_row_similarity [0.0, 0.1, 0.2, 0.4, 0.5, 0.8]



In [35]:
features = ['mem_range']
# features_values = [['[256-512]','[512-1024]','[1024-2048]']]
features_values = [['[1024-2048]']]
filter_data_performance(group_system_best, 'Grace-H100', features, features_values, [0,100])

Filter applied to data on device Grace-H100: 
mem_range ['[1024-2048]']

Performance range of Grace-H100 for these matrices: 76.49 - 605.5
36 / 7553 matrices achieve performance of [0, 100] GFLOPs
Implementations of performance in range  [0, 100] {'cu-CSR'}
[0, 100] -> avg_nz_row [5.0]
[0, 100] -> skew_coeff [0.0, 100.0, 1000.0, 9990.0, 10000.0]
[0, 100] -> avg_num_neighbours [0.0, 0.4]
[0, 100] -> cross_row_similarity [0.0, 0.1]



In [36]:
features = ['mem_range']
# features_values = [['[256-512]','[512-1024]','[1024-2048]']]
features_values = [['[1024-2048]']]
filter_data_performance(group_system_best, 'Grace-H100', features, features_values, [0,150])

Filter applied to data on device Grace-H100: 
mem_range ['[1024-2048]']

Performance range of Grace-H100 for these matrices: 76.49 - 605.5
186 / 7553 matrices achieve performance of [0, 150] GFLOPs
Implementations of performance in range  [0, 150] {'cu-CSR'}
[0, 150] -> avg_nz_row [5.0, 10.0]
[0, 150] -> skew_coeff [0.0, 100.0, 1000.0, 9990.0, 10000.0]
[0, 150] -> avg_num_neighbours [0.0, 0.4, 0.5, 0.7, 0.8, 0.9, 1.1]
[0, 150] -> cross_row_similarity [0.0, 0.1, 0.4, 0.5]



---

In [38]:
def select_region2(system, what, group_data):
    # plot synthetic dataset only # used to be 'friends' != True here... but no point in it. move on with sth basic
    select_criterion = (group_data['Arch'] != '')
    if(system!=''):
        select_criterion = select_criterion & (group_data['System']==system)
    select_str = ''

    # features = ['A_mem_footprint','avg_nz_row', 'skew_coeff', 'avg_num_neighbours', 'cross_row_similarity']
    features = ['avg_nz_row', 'skew_coeff', 'regularity']
    for feat in features:
        if(feat=='regularity'):
            if(what == 'good'):
                irr_val_list = ['LL']
                if(system=='Alveo-U280'):
                    irr_val_list = ['LS']
            # elif(what == 'bad'):
            #     irr_val_list = ['SS']
            #     if(system=='Alveo-U280'):
            #         irr_val_list = ['SL']
            elif(what == 'all'):
                irr_val_list = ['SS', 'SM', 'SL', 'MS', 'MM', 'ML', 'LS', 'LM', 'LL']

            select_str = select_str + feat + '=['+ ','.join(irr_val_list) +']' + '  '
            for irr_val in irr_val_list:
                if(irr_val == irr_val_list[0]):
                    select_criterion_irr = (group_data[feat] == irr_val)
                else:
                    select_criterion_irr = select_criterion_irr | (group_data[feat] == irr_val)
            select_criterion = select_criterion & select_criterion_irr
        else:
            if(what == 'good'):
                if(feat == 'avg_nz_row'):
                    min_feat, max_feat = 45, 510                        
                if(feat == 'skew_coeff'):
                    min_feat, max_feat = 0, 50
            if(what == 'all'):
                if(feat == 'avg_nz_row'):
                    min_feat, max_feat = 0, 510
                if(feat == 'skew_coeff'):
                    prmin_feat, max_feat = 0, 180000
            select_str = select_str + feat + '=['+str(min_feat)+'-'+str(max_feat)+']' + ' '
            select_criterion = select_criterion & (group_data[feat] >= min_feat) & (group_data[feat] <= max_feat)
    return select_criterion, select_str


def select_region3(system, what, group_data, fpga_thres = 64):
    # plot synthetic dataset only # used to be 'friends' != True here... but no point in it. move on with sth basic
    select_criterion = (group_data['Arch'] != '')
    if(system!=''):
        select_criterion = select_criterion & (group_data['System']==system)
    select_str = ''

    feat = 'A_mem_footprint'
    if(what == 'big'):
        if(system=='Tesla-P100'):
            min_feat, max_feat = 64, 2048
        if(system=='Tesla-V100'):
            min_feat, max_feat = 64, 2048
        if(system=='Tesla-A100'):
            min_feat, max_feat = 64, 2048
        if(system=='Grace-H100'):
            min_feat, max_feat = 64, 2048
        if(system=='AMD-EPYC-64'):
            min_feat, max_feat = 256, 2048
        if(system=='AMD-EPYC-24'):
            min_feat, max_feat = 128, 2048
        if(system=='ARM-NEON'):
            min_feat, max_feat = 64, 2048
        if(system=='ARM-GRACE'):
            min_feat, max_feat = 128, 2048
        if(system=='INTEL-XEON'):
            min_feat, max_feat = 32, 2048
        if(system=='INTEL-ICY'):
            min_feat, max_feat = 64, 2048
        if(system=='IBM-POWER9'):
            min_feat, max_feat = 64, 2048
        if(system=='Alveo-U280'):
            min_feat, max_feat = fpga_thres, 2048
    if(what == 'small'):
        if(system=='Tesla-P100'):
            min_feat, max_feat = 8, 64
        if(system=='Tesla-V100'):
            min_feat, max_feat = 8, 64
        if(system=='Tesla-A100'):
            min_feat, max_feat = 8, 64
        if(system=='Grace-H100'):
            min_feat, max_feat = 8, 64
        if(system=='AMD-EPYC-64'):
            min_feat, max_feat = 8, 256
        if(system=='AMD-EPYC-24'):
            min_feat, max_feat = 8, 128
        if(system=='ARM-NEON'):
            min_feat, max_feat = 8, 64
        if(system=='ARM-GRACE'):
            min_feat, max_feat = 8, 128
        if(system=='INTEL-XEON'):
            min_feat, max_feat = 8, 32
        if(system=='INTEL-ICY'):
            min_feat, max_feat = 8, 32
        if(system=='IBM-POWER9'):
            min_feat, max_feat = 8, 64
        if(system=='Alveo-U280'):
            min_feat, max_feat = 8, fpga_thres
    if(what == 'all'):
        min_feat, max_feat = 0, 2048

    select_str = select_str + feat + '=['+str(min_feat)+'-'+str(max_feat)+']' + ' '
    select_criterion = select_criterion & (group_data[feat] >= min_feat) & (group_data[feat] <= max_feat)

    return select_criterion, select_str


###################################################################################################################
###################################################################################################################
def lighten_color(color, amount=1.6):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])


def plot_dev_perf(feature, pcg, what, group_data, group_metadata=[]):
    if(feature == 'A_mem_footprint'):
        system2 = 'Feature Impact-all_and_good_together'
    else:
        system2 = 'Feature Impact-small_and_big_together'

    plot_x_axis = feature
    if(feature=='A_mem_footprint'):
        plot_x_axis = 'mem_range'
    if(feature=='avg_nz_row'):
        plot_x_axis = 'anr_categ'
    if(feature=='skew_coeff'):
        plot_x_axis = 'skew_categ'

    plot_y_axis = 'gflops'

    plot_id = system2
    select_str = feature

    ctr = 0
    fig, axs = plt.subplots(len(ranges_dev))
    fig.subplots_adjust(left=.15, bottom=.17*(2/6)*(4/3), right=.99, top=.99)
    for system in ranges_dev:
        select_criterion_tmp, select_str_tmp = select_region(system, feature, pcg, what, group_data, group_metadata=[])
        group_system_select_tmp = group_data[select_criterion_tmp]

        group_system_select_tmp_sorted = group_system_select_tmp
        if plot_x_axis == 'mem_range':
            cat_it = CategoricalDtype(ranges_memr, ordered=True)
        if plot_x_axis == 'regularity':
            cat_it = CategoricalDtype(ranges_regularity, ordered=True)
        if plot_x_axis == 'anr_categ':
            cat_it = CategoricalDtype(ranges_anr, ordered=True)
        if plot_x_axis == 'skew_categ':
            cat_it = CategoricalDtype(ranges_skew, ordered=True)

        pd.options.mode.chained_assignment = None # https://stackoverflow.com/a/49729413
        group_system_select_tmp_sorted[plot_x_axis] = group_system_select_tmp_sorted[plot_x_axis].astype(cat_it)
        group_system_select_tmp_sorted = group_system_select_tmp_sorted.sort_values(plot_x_axis)

        # for "all" vs "good", we do not want to change colours
        # for "small" vs "large", we want to change them (make "small" a bit lighter, by "var")
        # make "small" vs "large"
        if('small' in what_list):
            var = 0.6
        else:
            var = 0.9

        # if len==3, in case we want to plot "all" vs "small" vs "big"
        color_sys = color_dictionary[system]
        if(len(what_list)==2):
            col_palette = list(map(lambda x: sns.desaturate(x,0.9),
                                   [lighten_color(color_sys, var), lighten_color(color_sys, 1.3)]))
        else:
            col_palette = list(map(lambda x: sns.desaturate(x,0.9),
                                   [color_sys, lighten_color(color_sys, 0.5), lighten_color(color_sys, 1.8)]))
        sns.set_palette(col_palette)
        sns.color_palette()

        sns_plot = sns.boxplot(fliersize = 0.3, linewidth=0.6, width=0.7, data=group_system_select_tmp, x=plot_x_axis, y=plot_y_axis, 
                               hue='mark_perf', hue_order=what_list, ax=axs[ctr])
                               #hue='System', hue_order=ranges_dev, ax=axs[ctr], dodge=False)

        axs[ctr].tick_params(axis='both', which='major', labelsize=font)
        axs[ctr].tick_params(axis='both', which='minor', labelsize=font)
        axs[ctr].set_ylabel('')

        if(plot_y_axis == 'gflops'):
            axs[ctr].set_ylim(y_limit_dictionary[system])

        # set x label only for last platform
        if(system == ranges_dev[-1]):  
            if(plot_x_axis == 'mem_range'):
                axs[ctr].set_xlabel('memory range (MB)', fontsize = font)
            if(plot_x_axis == 'anr_categ'):
                axs[ctr].set_xlabel('average number of nonzeros per row', fontsize = font)
            if(plot_x_axis == 'skew_categ'):
                axs[ctr].set_xlabel('skew coefficient', fontsize = font)
            if(plot_x_axis == 'regularity'):
                axs[ctr].set_xlabel('regularity ( cross row similarity | average number of neighbors )', fontsize = font)
        else:
            axs[ctr].set_xlabel('')
        
        # remove x ticks for all platforms except for last
        if(system != ranges_dev[-1]):
            axs[ctr].set_xticks([])
            axs[ctr].set_xticklabels([])

        # set legend that was used in previous plot....because here it would show "all" vs "good"
        if(system == ranges_dev[0]):
            axs[ctr].legend([i for i in handles], [i for i in labels], loc='upper center', bbox_to_anchor=(0.5, 1.43), ncol=len(ranges_dev), frameon=False)
        else:
            axs[ctr].get_legend().remove()

        plt.xticks(rotation=0)
        if(plot_x_axis == 'mem_range'):
            plt.xticks(rotation=17.5)
            
        axs[ctr].text( 0.04, 0.85, ranges_dev[ctr], fontsize=font-1, transform = axs[ctr].transAxes, )

        ctr+=1

    select_str_id = select_str.replace('=', 'eq').replace('<', 'l').replace('>', 'g').replace(', ', '_').replace(',', '_').replace('\n', '_').replace(' ', '-')
    if('-performance_' in select_str_id):
        select_str_id = select_str_id.split('-performance_')[0]
    else:
        select_str_id = select_str_id.split('__')[0]
    
    # because I needed to rotate the x-axis labels a little bit for new font size, need to increase height slightly
    if(plot_x_axis == 'mem_range'):
        fig.set_size_inches(3 * width, 1.0*len(ranges_dev) * height)
        fig.text(0.00, 0.55, 'GFLOPs', va='center', rotation='vertical', fontsize=font)
    else:
        fig.set_size_inches(3 * width, 1.0*len(ranges_dev) * height)
        fig.text(0.00, 0.53, 'GFLOPs', va='center', rotation='vertical', fontsize=font)
        
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)
    fig_name = './Plots/%s_Select-%s_x-%s_y-%s-NEW.pdf' % (plot_id, select_str_id, plot_x_axis, plot_y_axis)
    plt.savefig(fig_name)
    plt.close()
    subprocess.run(['pdfcrop', fig_name], stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
    subprocess.run(['mv', fig_name.replace('.pdf', '-crop.pdf'), fig_name])

###################################################################################################################
###################################################################################################################

group_data = group_system_best
# ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'Grace-H100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'INTEL-XEON', 'INTEL-ICY', 'ARM-NEON', 'ARM-GRACE', 'IBM-POWER9', 'Alveo-U280']
ranges_dev = ['Tesla-A100', 'Grace-H100', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE']

change_colors(ranges_dev, color_dictionary)

what_list = ['all', 'good']
########################    
group_data_system_list=[]
for system in ranges_dev:
    select_str_all, tmp = select_region2(system, 'all', group_data)
    select_str_good, tmp = select_region2(system, 'good', group_data)
    group_data_all_system = group_data[select_str_all]
    group_data_all_system['mark_perf'] = 'all'
    group_data_good_system = group_data[select_str_good]
    group_data_good_system['mark_perf'] = 'good'
    group_data_system = pd.concat([group_data_all_system, group_data_good_system])
    # plot_dev_perf3(group_data_system, system)
    group_data_system_list.append(group_data_system)    
group_data_system_list = pd.concat(group_data_system_list)
plot_dev_perf('A_mem_footprint', 10, 'all', group_data_system_list)

########################

# what_list = ['all', 'small', 'big']
what_list = ['small', 'big']

# fpga_thres = 64 # fpga_thres = 128 # fpga_thres = 256 # fpga_thres = 512
fpga_thres = 256

group_data_system_list=[]
for system in ranges_dev:
    select_str_small, tmp = select_region3(system, 'small', group_data, fpga_thres = fpga_thres )
    select_str_big, tmp = select_region3(system, 'big', group_data, fpga_thres = fpga_thres )
    select_str_all, tmp = select_region3(system, 'all', group_data, fpga_thres = fpga_thres )
    group_data_small_system = group_data[select_str_small]
    group_data_small_system['mark_perf'] = 'small'
    group_data_big_system = group_data[select_str_big]
    group_data_big_system['mark_perf'] = 'big'
    group_data_all_system = group_data[select_str_all]
    group_data_all_system['mark_perf'] = 'all'

    if(len(what_list)==2):
        group_data_system = pd.concat([group_data_small_system, group_data_big_system])
    else:
        group_data_system = pd.concat([group_data_small_system, group_data_big_system, group_data_all_system])

    # plot_dev_perf3(group_data_system, system)
    group_data_system_list.append(group_data_system)    
group_data_system_list = pd.concat(group_data_system_list)

features = ['avg_nz_row', 'skew_coeff', 'regularity']
# features = ['avg_nz_row'] # features = ['skew_coeff'] # features = [ 'regularity']

for feat in features:
    plot_dev_perf(feat, 10, 'all', group_data_system_list)


---
# print distribution of each feature for each device

In [41]:
group_data = group_system_best
# ranges_dev = ['Tesla-V100', 'Tesla-A100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'IBM-POWER9', 'Alveo-U280']
# ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'Grace-H100', 
#               'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
ranges_dev = ['Grace-H100']
what_list = ['all', 'good']
########################    
group_data_system_list=[]
for system in ranges_dev:
    # print('SYSTEM : ', system)
    select_str_all, tmp = select_region2(system, 'all', group_data)
    select_str_good, tmp = select_region2(system, 'good', group_data)
    group_data_all_system = group_data[select_str_all]
    group_data_all_system['mark_perf'] = 'all'
    group_data_good_system = group_data[select_str_good]
    group_data_good_system['mark_perf'] = 'good'
    group_data_system = pd.concat([group_data_all_system, group_data_good_system])
    # plot_dev_perf3(group_data_system, system)
    group_data_system_list.append(group_data_system)    
group_data_system_list = pd.concat(group_data_system_list)
########################

def print_distribution(feature, feat_categ, ranges_feature, group_system_best):
    total = group_system_best.shape[0]
    print(feature + ' distribution in dataset')
    for val in ranges_feature:
        group_system_best_feat = group_system_best[group_system_best[feat_categ]==val]
        cnt = group_system_best_feat.shape[0]
        print(val, '\t', cnt,'/', total,'\t',np.round(100*cnt/total,2), '%\t', np.round(group_system_best_feat['gflops'].median(),2),'GFLOPs')

print_distribution('mem_range', 'mem_range', ranges_memr, group_system_best)
print_distribution('avg_nz_row', 'anr_categ', ranges_anr, group_system_best)
print_distribution('skew_coeff', 'skew_categ', ranges_skew, group_system_best)
print_distribution('regularity(cross_row_similarity / avg_num_neighbours)', 'regularity', ranges_regularity, group_system_best)

print('---------------')
for mem_range in ranges_memr:
    group_systemaki = group_data_good_system[group_data_good_system['mem_range']==mem_range]
    print('good', mem_range, round(group_systemaki['gflops'].median(),2))
    group_systemaki = group_data_all_system[group_data_all_system['mem_range']==mem_range]
    print('all ', mem_range, round(group_systemaki['gflops'].median(),2))
    print('-')


mem_range distribution in dataset
[4-8] 	 6114 / 168873 	 3.62 %	 37.37 GFLOPs
[8-16] 	 21058 / 168873 	 12.47 %	 57.02 GFLOPs
[16-32] 	 24745 / 168873 	 14.65 %	 66.18 GFLOPs
[32-64] 	 8127 / 168873 	 4.81 %	 64.75 GFLOPs
[64-128] 	 3103 / 168873 	 1.84 %	 41.92 GFLOPs
[128-256] 	 21571 / 168873 	 12.77 %	 31.05 GFLOPs
[256-512] 	 25035 / 168873 	 14.82 %	 18.52 GFLOPs
[512-1024] 	 22184 / 168873 	 13.14 %	 18.41 GFLOPs
[1024-2048] 	 33650 / 168873 	 19.93 %	 17.94 GFLOPs
avg_nz_row distribution in dataset
[0-15] 	 56232 / 168873 	 33.3 %	 25.23 GFLOPs
[15-40] 	 27576 / 168873 	 16.33 %	 40.18 GFLOPs
[40-75] 	 28914 / 168873 	 17.12 %	 40.73 GFLOPs
[75-150] 	 28617 / 168873 	 16.95 %	 43.13 GFLOPs
[150-510] 	 27534 / 168873 	 16.3 %	 48.57 GFLOPs
skew_coeff distribution in dataset
[0-1.5] 	 15440 / 168873 	 9.14 %	 53.44 GFLOPs
[1.5-50] 	 37272 / 168873 	 22.07 %	 35.56 GFLOPs
[50-250] 	 47670 / 168873 	 28.23 %	 37.89 GFLOPs
[250-3000] 	 41135 / 168873 	 24.36 %	 34.4 GFLOPs
[3000-10

---
# Compare wins of GPU over CPU for "favorable" memory footprint (depending on cache size of examined CPU)

In [42]:
def compare_GPU_vs_CPU(system1, system2, mem_ranges):
    group_system_best_iso = group_system_best[(group_system_best['System'] == system1) \
                                             |(group_system_best['System'] == system2)]
    
    mask = np.logical_or.reduce([(group_system_best_iso['mem_range'] == mem_range) for mem_range in mem_ranges])
    group_system_best_iso = group_system_best_iso[mask]
    
    # group_system_best_iso = group_system_best_iso[(group_system_best_iso['mem_range'] == '[32-64]') | \
    #                                              (group_system_best_iso['mem_range'] == '[64-128]')]
    print(group_system_best_iso.shape)
    # print(group_system_best_iso.columns)

    group_system_best_iso = group_system_best_iso.round({"A_mem_footprint":2})
    group_system_best_iso = group_system_best_iso.round({"avg_nz_row":2})
    group_system_best_iso = group_system_best_iso.round({"skew_coeff":2})

    group_system_best_iso2 = group_system_best_iso.groupby(['A_mem_footprint', 'avg_nz_row', 'skew_coeff', 'regularity'], as_index = False, observed = True)
    reslist_gflops, reslist_energy_efficiency = [], []
    reslist_gflops2, reslist_energy_efficiency2 = [], []
    cnt2, cnt3 = 0, 0
    for desc, experiment in group_system_best_iso2:
        # there is a problem that some matrices have different feature values between Systems
        # have to omit those that 
        if(len(set(experiment['System']))>1):
            # print(desc, experiment)
            best_device_gflops = experiment['System'].iloc[experiment['gflops'].argmax()]
            best_device_energy_efficiency = experiment['System'].iloc[experiment['energy_efficiency'].argmax()]
            # print(best_device_gflops) # print(best_device_energy_efficiency)
            outrow_gflops = experiment[experiment['System'] == best_device_gflops]
            outrow_energy_efficiency = experiment[experiment['System'] == best_device_energy_efficiency]
            # print(outrow_gflops) # print(outrow_energy_efficiency)
            # break
            reslist_gflops.append(outrow_gflops.values.tolist()[0])
            reslist_energy_efficiency.append(outrow_energy_efficiency.values.tolist()[0])
        # else:
        #     cnt2+=1
    # print(cnt2, 'could not be included (Alveo shit)')
    # print()

    group_system_best_gflops = pd.DataFrame(reslist_gflops)#, columns = header_names)
    group_system_best_energy_efficiency = pd.DataFrame(reslist_energy_efficiency)#, columns = header_names)
    print('gflops wins %')
    shape_wins = group_system_best_gflops.shape[0]
    print(system1, round(group_system_best_gflops[group_system_best_gflops[28]==system1].shape[0]/shape_wins*100,2))
    print(system2, round(group_system_best_gflops[group_system_best_gflops[28]==system2].shape[0]/shape_wins*100,2))
    print('---')
    print('energy efficiency wins %')
    print(system1, round(group_system_best_energy_efficiency[group_system_best_energy_efficiency[28]==system1].shape[0]/shape_wins*100,2))
    print(system2, round(group_system_best_energy_efficiency[group_system_best_energy_efficiency[28]==system2].shape[0]/shape_wins*100,2))
    # print('Alveo-U280', round(group_system_best_energy_efficiency[group_system_best_energy_efficiency[28]=='Alveo-U280'].shape[0]/shape_wins*100,2))


# wins of AMD-EPYC-64 over Tesla-V100 for [64-256] MB matrices

In [43]:
system1 = 'Tesla-V100'
system2 = 'AMD-EPYC-64'
mem_ranges = ['[64-128]', '[128-256]']
compare_GPU_vs_CPU(system1, system2, mem_ranges)

(3971, 38)
gflops wins %
Tesla-V100 35.67
AMD-EPYC-64 64.33
---
energy efficiency wins %
Tesla-V100 99.42
AMD-EPYC-64 0.58


# wins of AMD-EPYC-64 over Tesla-A100 for [64-256] MB matrices

In [46]:
system1 = 'Tesla-A100'
system2 = 'AMD-EPYC-64'
mem_ranges = ['[64-128]', '[128-256]']
compare_GPU_vs_CPU(system1, system2, mem_ranges)

(4311, 38)
gflops wins %
Tesla-A100 100.0
AMD-EPYC-64 0.0
---
energy efficiency wins %
Tesla-A100 100.0
AMD-EPYC-64 0.0


# wins of AMD-EPYC-24 over Tesla-V100 for [32-128] MB matrices

In [47]:
system1 = 'Tesla-V100'
system2 = 'AMD-EPYC-24'
mem_ranges = ['[32-64]', '[64-128]']
compare_GPU_vs_CPU(system1, system2, mem_ranges)

(1801, 38)
gflops wins %
Tesla-V100 58.28
AMD-EPYC-24 41.72
---
energy efficiency wins %
Tesla-V100 98.01
AMD-EPYC-24 1.99


# wins of ARM-NEON over Tesla-V100 for [32-128] MB matrices

In [48]:
system1='Tesla-V100'
system2='ARM-NEON'
mem_ranges = ['[32-64]', '[64-128]']
compare_GPU_vs_CPU(system1, system2, mem_ranges)

(1801, 38)
gflops wins %
Tesla-V100 29.8
ARM-NEON 70.2
---
energy efficiency wins %
Tesla-V100 26.49
ARM-NEON 73.51


# wins of A100 over everyone

In [49]:
# group_system_best_iso = group_system_best[(group_system_best['System'] == 'Tesla-A100') \
#                                          |(group_system_best['System'] == 'Tesla-V100') \
#                                          |(group_system_best['System'] == 'AMD-EPYC') \
#                                          |(group_system_best['System'] == 'ARM-NEON')]

# group_system_best_iso = group_system_best_iso[(group_system_best_iso['mem_range'] == '[32-64]') | \
#                                              (group_system_best_iso['mem_range'] == '[64-128]') | \
#                                              (group_system_best_iso['mem_range'] == '[128-256]')]
# print(group_system_best_iso.shape)
# # print(group_system_best_iso.columns)

# group_system_best_iso = group_system_best_iso.round({"A_mem_footprint":2})
# group_system_best_iso = group_system_best_iso.round({"avg_nz_row":2})
# group_system_best_iso = group_system_best_iso.round({"skew_coeff":2})

# group_system_best_iso2 = group_system_best_iso.groupby(['A_mem_footprint', 'avg_nz_row', 'skew_coeff', 'regularity'], as_index = False)
# reslist_gflops, reslist_energy_efficiency = [], []
# reslist_gflops2, reslist_energy_efficiency2 = [], []
# cnt2, cnt3 = 0, 0
# for desc, experiment in group_system_best_iso2:
#     # there is a problem that some matrices have different feature values between Systems
#     # have to omit those that 
#     if(len(set(experiment['System']))>1):
#         # print(desc, experiment)
#         best_device_gflops = experiment['System'].iloc[experiment['gflops'].argmax()]
#         best_device_energy_efficiency = experiment['System'].iloc[experiment['energy_efficiency'].argmax()]
#         # print(best_device_gflops) # print(best_device_energy_efficiency)
#         outrow_gflops = experiment[experiment['System'] == best_device_gflops]
#         outrow_energy_efficiency = experiment[experiment['System'] == best_device_energy_efficiency]
#         # print(outrow_gflops) # print(outrow_energy_efficiency)
#         # break
#         reslist_gflops.append(outrow_gflops.values.tolist()[0])
#         reslist_energy_efficiency.append(outrow_energy_efficiency.values.tolist()[0])
#     else:
#         cnt2+=1
# print(cnt2, 'could not be included (Alveo shit)')
# print()

# group_system_best_gflops = pd.DataFrame(reslist_gflops)#, columns = header_names)
# group_system_best_energy_efficiency = pd.DataFrame(reslist_energy_efficiency)#, columns = header_names)
# print('gflops wins %')
# shape_wins = group_system_best_gflops.shape[0]
# print('Tesla-A100', round(group_system_best_gflops[group_system_best_gflops[28]=='Tesla-A100'].shape[0]/shape_wins*100,2))
# print('Tesla-V100', round(group_system_best_gflops[group_system_best_gflops[28]=='Tesla-V100'].shape[0]/shape_wins*100,2))
# print('AMD-EPYC', round(group_system_best_gflops[group_system_best_gflops[28]=='AMD-EPYC'].shape[0]/shape_wins*100,2))
# print('ARM-NEON', round(group_system_best_gflops[group_system_best_gflops[28]=='ARM-NEON'].shape[0]/shape_wins*100,2))
# # print('---')
# # print('energy efficiency wins %')
# # print('Tesla-V100', round(group_system_best_energy_efficiency[group_system_best_energy_efficiency[28]=='Tesla-V100'].shape[0]/shape_wins*100,2))
# # print('ARM-NEON', round(group_system_best_energy_efficiency[group_system_best_energy_efficiency[28]=='ARM-NEON'].shape[0]/shape_wins*100,2))
# # print('Alveo-U280', round(group_system_best_energy_efficiency[group_system_best_energy_efficiency[28]=='Alveo-U280'].shape[0]/shape_wins*100,2))

# Compare ARM and AMD-64 vs A100 for specific memory range

In [50]:
a = group_system_best[group_system_best['System']=='ARM-NEON']
a = a[(a['mem_range'] == '[32-64]') | (a['mem_range'] == '[64-128]')]
b = group_system_best[group_system_best['System']=='Tesla-A100']
b = b[(b['mem_range'] == '[32-64]') | (b['mem_range'] == '[64-128]')]
print('ARM % of A100 performance in 32-128 range\t->', round(a['gflops'].median()/b['gflops'].median()*100),'%')


a = group_system_best[group_system_best['System']=='AMD-EPYC-64']
a = a[(a['mem_range'] == '[64-128]') | (a['mem_range'] == '[128-256]')]
b = group_system_best[group_system_best['System']=='Tesla-A100']
b = b[(b['mem_range'] == '[64-128]') | (b['mem_range'] == '[128-256]')]
print('AMD-EPYC-64 % of A100 performance in 64-256 range\t->', round(a['gflops'].median()/b['gflops'].median()*100),'%')

ARM % of A100 performance in 32-128 range	-> 73 %
AMD-EPYC-64 % of A100 performance in 64-256 range	-> 61 %


---
# plot median performance per feature range

In [51]:
def print_distribution_col100(feature, feat_categ, ranges_feature, group_system_best, verbose = True, system_name=''):
    total = group_system_best.shape[0]
    if(verbose):
        print(feature + ' distribution in dataset')
    ndf_row = [system_name]
    for val in ranges_feature:
        group_system_best_feat = group_system_best[group_system_best[feat_categ]==val]
        cnt = group_system_best_feat.shape[0]
        new_string = str(np.round(group_system_best_feat['gflops'].median(),2))# + " GF"
        if(verbose):
            print(val, '\t', cnt,'/', total,'\t', new_string)
        ndf_row.append(new_string)
    return ndf_row

group_data = group_system_best
# group_data = groupreps

ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'Grace-H100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
research_formats = ['Merge-CSR', 'SparseX', 'SELL-C-s', 'CSR5']

research_only = False
# research_only = True

features = ['mem_range', 'anr_categ', 'skew_categ', 'regularity']
rangess = [ranges_memr, ranges_anr, ranges_skew, ranges_regularity]


for cnttt in range(len(features)):
    feat, ranges_feat = features[cnttt], rangess[cnttt]
    print(feat)
    
    new_df_list = []    
    group_data_system_list=[]
    for system in ranges_dev:
        formats = [x.split("\n")[1] for x in ranges_impl_arch_backup if 
                   ((x.split(" )")[0].split("( ")[1] == system) and 
                    (x.split("\n")[1] in set(group_data[group_data['System']==x.split(" )")[0].split("( ")[1]]['implementation'])))]

        if(research_only):
            formats = [ i if i in research_formats else '' for i in formats]
            while('' in formats):
                formats.remove('')

        group_data_sys = group_data[group_data['System']==system]

        ndf_row = print_distribution_col100(feat, feat, ranges_feat, group_data_sys, verbose=False, system_name=system)
        new_df_list.append(ndf_row)

    new_df = pd.DataFrame(new_df_list, columns = ['System'] + ranges_feat)
    if(new_df.shape[0]>0):
        new_df.set_index('System', inplace=True)
        new_df = new_df.T
        display(new_df)


mem_range


System,Tesla-P100,Tesla-V100,Tesla-A100,Grace-H100,AMD-EPYC-24,AMD-EPYC-64,ARM-NEON,ARM-GRACE,INTEL-XEON,INTEL-ICY,IBM-POWER9,Alveo-U280
[4-8],25.61,38.57,40.58,59.25,49.24,36.68,36.2,56.73,26.03,40.59,23.23,16.9
[8-16],36.32,51.7,85.49,127.23,67.6,66.58,74.38,108.78,27.05,47.5,25.62,19.57
[16-32],44.38,69.75,114.55,208.73,68.27,88.97,101.2,164.48,22.09,28.03,26.28,19.6
[32-64],43.92,75.35,122.69,251.58,64.72,82.73,107.35,184.96,17.23,23.94,25.67,15.6
[64-128],47.54,99.69,145.76,376.62,34.73,85.54,22.1,148.97,10.43,11.35,14.93,14.9
[128-256],52.13,103.7,155.58,394.26,15.56,95.52,20.63,98.05,10.39,11.14,15.38,16.2
[256-512],51.43,109.78,158.17,414.99,12.59,13.12,17.33,57.83,9.42,10.37,13.51,19.3
[512-1024],49.96,108.04,156.09,419.48,11.31,16.67,15.01,48.63,8.71,9.86,12.36,21.2
[1024-2048],46.43,104.74,154.22,426.24,11.15,14.88,13.52,45.43,8.12,9.63,11.54,22.8


anr_categ


System,Tesla-P100,Tesla-V100,Tesla-A100,Grace-H100,AMD-EPYC-24,AMD-EPYC-64,ARM-NEON,ARM-GRACE,INTEL-XEON,INTEL-ICY,IBM-POWER9,Alveo-U280
[0-15],26.51,50.61,109.28,251.45,10.09,35.45,13.67,54.59,7.88,9.07,12.55,6.76
[15-40],nan,73.18,129.13,348.05,12.41,61.51,17.66,60.79,10.05,10.62,17.66,15.9
[40-75],42.64,93.78,143.15,390.71,14.76,24.33,20.55,61.22,11.2,12.35,20.66,20.0
[75-150],47.67,106.07,159.3,406.42,16.27,24.71,22.48,66.23,11.85,13.52,22.77,21.5
[150-510],55.14,119.31,177.57,429.26,17.82,25.16,26.47,75.02,12.97,15.52,25.37,27.0


skew_categ


System,Tesla-P100,Tesla-V100,Tesla-A100,Grace-H100,AMD-EPYC-24,AMD-EPYC-64,ARM-NEON,ARM-GRACE,INTEL-XEON,INTEL-ICY,IBM-POWER9,Alveo-U280
[0-1.5],44.46,70.1,105.85,176.11,67.73,78.08,78.11,126.23,23.59,38.35,28.43,23.02
[1.5-50],41.68,77.46,136.41,335.08,14.27,25.31,18.81,58.82,9.36,11.12,13.12,18.4
[50-250],47.32,86.93,134.01,345.2,16.55,36.09,23.27,64.3,11.83,13.7,19.31,20.9
[250-3000],47.15,90.26,134.33,350.44,15.19,25.46,20.49,57.37,10.98,12.63,17.72,18.1
[3000-10000],39.58,86.33,133.12,356.49,12.8,19.02,16.88,52.35,8.96,10.12,14.53,9.28


regularity


System,Tesla-P100,Tesla-V100,Tesla-A100,Grace-H100,AMD-EPYC-24,AMD-EPYC-64,ARM-NEON,ARM-GRACE,INTEL-XEON,INTEL-ICY,IBM-POWER9,Alveo-U280
SS,35.34,74.25,114.04,257.71,14.04,25.42,17.92,52.95,9.0,11.46,10.96,20.4
SM,35.08,72.48,125.75,332.18,13.94,24.92,18.25,55.26,9.0,10.86,12.31,17.2
SL,43.02,74.55,141.88,389.42,15.55,24.85,21.08,59.44,10.98,12.53,19.54,8.7
MS,40.75,79.79,126.85,320.82,14.23,25.51,18.55,57.34,9.82,11.73,12.54,20.6
MM,41.32,74.56,132.94,364.24,13.84,34.7,18.86,60.42,10.04,11.28,15.58,18.09
ML,47.85,81.81,148.84,399.54,16.58,36.6,23.05,68.43,11.66,13.69,23.33,11.9
LS,49.89,88.45,148.95,395.5,16.54,28.75,22.27,62.9,11.3,13.37,20.77,20.5
LM,52.27,87.88,144.83,390.66,17.83,42.33,26.0,73.95,11.95,13.94,22.22,19.23
LL,52.2,85.55,143.29,324.26,54.23,64.46,38.11,98.63,15.18,20.52,25.6,16.48


---
# plot median performance per format per feature range
# for ALL matrices of dataset per format (not only best execution of matrix)

In [52]:
def print_distribution_col100(feature, feat_categ, ranges_feature, group_system_best, verbose = True, format_name=''):
    total = group_system_best.shape[0]
    if(verbose):
        print(feature + ' distribution in dataset')
    ndf_row = [format_name]
    for val in ranges_feature:
        group_system_best_feat = group_system_best[group_system_best[feat_categ]==val]
        cnt = group_system_best_feat.shape[0]
        new_string = str(np.round(group_system_best_feat['gflops'].median(),2))# + " GF"
        if(verbose):
            print(val, '\t', cnt,'/', total,'\t', new_string)
        ndf_row.append(new_string)
    return ndf_row

# group_data = group_system_best
group_data = groupreps

ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'Grace-H100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
research_formats = ['Merge-CSR', 'SparseX', 'SELL-C-s', 'CSR5']

research_only = False
# research_only = True

features = ['mem_range', 'anr_categ', 'skew_categ', 'regularity']
rangess = [ranges_memr, ranges_anr, ranges_skew, ranges_regularity]

for cnttt in range(len(features)):
    feat, ranges_feat = features[cnttt], rangess[cnttt]
    print(feat)
    
    group_data_system_list=[]
    for system in ranges_dev:
        print('\n'+system)
        formats = [x.split("\n")[1] for x in ranges_impl_arch_backup if 
                   ((x.split(" )")[0].split("( ")[1] == system) and 
                    (x.split("\n")[1] in set(group_data[group_data['System']==x.split(" )")[0].split("( ")[1]]['implementation'])))]

        if(research_only):
            formats = [ i if i in research_formats else '' for i in formats]
            while('' in formats):
                formats.remove('')

        group_data_sys = group_data[group_data['System']==system]

        new_df_list = []
        for f in formats:
            group_data_sys_f = group_data_sys[group_data_sys['implementation'] == f]
            # print(system, '\t', f, '\t', group_data_sys_f.shape[0])


            # group_data_sys_f = group_data_sys_f.round({'avg_nz_row':0, 'skew_coeff':-2})
            # print('avg_nz_row', '\t', sorted(set(group_data_sys_f['avg_nz_row'])))
            # print('skew_coeff', '\t', sorted(set(group_data_sys_f['skew_coeff'])))
            # print('regularity', '\t', sorted(set(group_data_sys_f['regularity'])))

            # feat, ranges_feat = features[cnttt], rangess[cnttt]
            ndf_row = print_distribution_col100(feat, feat, ranges_feat, group_data_sys_f, verbose=False, format_name = f)
            new_df_list.append(ndf_row)

        new_df = pd.DataFrame(new_df_list, columns = ['Format'] + ranges_feat)
        if(new_df.shape[0]>0):
            new_df.set_index('Format', inplace=True)
            new_df = new_df.T
            display(new_df)


mem_range

Tesla-P100


Format,cu-COO,cu-CSR,cu-HYB,CSR5
[4-8],25.61,20.02,10.85,16.16
[8-16],35.74,27.52,18.75,31.39
[16-32],43.25,36.42,24.61,39.27
[32-64],38.18,29.04,37.53,33.3
[64-128],44.12,46.06,31.29,38.85
[128-256],43.76,36.34,42.71,41.12
[256-512],42.15,37.81,42.62,41.15
[512-1024],40.53,33.95,42.18,39.27
[1024-2048],39.9,32.54,38.01,35.96



Tesla-V100


Format,cu-COO,cu-CSR,cu-HYB,CSR5
[4-8],38.44,32.8,20.71,27.87
[8-16],48.07,43.88,27.38,47.13
[16-32],63.32,55.14,39.52,66.97
[32-64],68.17,53.27,63.55,68.62
[64-128],85.32,97.63,50.81,96.46
[128-256],86.2,62.57,87.64,97.84
[256-512],87.64,68.93,92.5,99.78
[512-1024],85.04,65.18,103.37,98.64
[1024-2048],78.92,66.09,85.73,85.9



Tesla-A100


...

Format,Naive-CSR,Vec-CSR,MKL-IE,CSR5,SparseX,Merge-CSR,SELL-C-s
SS,10.36,10.43,9.92,10.07,10.14,10.65,7.62
SM,10.09,10.16,9.92,9.97,8.84,10.32,7.57
SL,11.52,11.67,11.37,11.18,10.93,11.67,8.49
MS,10.69,10.91,10.26,10.49,10.18,11.09,8.23
MM,10.81,10.89,10.43,10.49,8.23,11.07,8.27
ML,11.83,12.11,11.99,11.69,11.29,12.0,9.22
LS,10.53,10.67,10.63,10.41,11.92,10.69,8.79
LM,11.26,11.36,11.08,11.07,12.1,11.48,9.2
LL,13.2,18.58,12.94,16.27,15.35,18.79,10.01



IBM-POWER9


Format,Naive-CSR,Bal-CSR,Vec-CSR,SparseX,Merge-CSR
SS,9.92,10.46,6.51,9.84,10.64
SM,11.38,11.92,6.88,9.15,12.21
SL,15.9,16.5,7.53,12.14,19.41
MS,11.42,11.81,6.95,10.48,12.26
MM,13.51,14.31,7.22,8.63,15.58
ML,18.93,20.17,7.79,12.43,23.17
LS,14.27,15.06,7.3,14.7,16.19
LM,17.73,18.88,7.62,14.32,21.47
LL,24.33,25.48,8.09,18.49,25.22



Alveo-U280


Format,Xilinx-lib
SS,20.4
SM,17.2
SL,8.7
MS,20.6
MM,18.09
ML,11.9
LS,20.5
LM,19.23
LL,16.48


# performance median (and percentage of matrices that format is best per feature range)
# (row100 - good)

In [54]:
def print_distribution_row100(feature, feat_categ, ranges_feature, group_system_best, verbose = True, format_name=''):
    total = group_system_best.shape[0]
    if(verbose):
        print(feature + ' distribution in dataset')
    ndf_row = [format_name]
    for val in ranges_feature:
        feature_range_total_matrices = group_system_best[group_system_best[feat_categ]==val].shape[0]
        group_data_sys_f = group_system_best[group_system_best['implementation'] == f]
        group_system_best_feat = group_data_sys_f[group_data_sys_f[feat_categ]==val]
        cnt = group_system_best_feat.shape[0]
        perc = np.round(100*cnt/feature_range_total_matrices,2)
        new_string = str(perc) + '% ('+ str(np.round(group_system_best_feat['gflops'].median(),2)) + ' GF)'
        if(verbose):
            if(perc>5):
                print(feature, val, group_system_best_feat.shape[0], 'matrices')
                group_system_best_feat = group_system_best_feat.round({'avg_nz_row':0, 'skew_coeff':-2})
                if(feature!='mem_range'):
                    print('mem_range', set(group_system_best_feat['mem_range']))
                if(feature!='anr_categ'):
                    print('avg_nz_row', sorted(set(group_system_best_feat['avg_nz_row'])))
                if(feature!='skew_categ'):
                    print('skew_coeff', sorted(set(group_system_best_feat['skew_coeff'])))
                if(feature!='regularity'):
                    print('regularity', set(group_system_best_feat['regularity']))
                    print('----------')
        if(verbose):
            print(val, '\t', cnt,'/', total,'\t', new_string)
        ndf_row.append(new_string)
    return ndf_row

group_data = group_system_best
# group_data = groupreps

# ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
# need to discard P100 for now...
ranges_dev = ['Tesla-V100', 'Tesla-A100', 'Grace-H100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
# ranges_dev = ['AMD-EPYC-24']
research_formats = ['Merge-CSR', 'SparseX', 'SELL-C-s', 'CSR5']

research_only = False

# cnn = 1
features = ['mem_range', 'anr_categ', 'skew_categ', 'regularity']
# features = [features[cnn]]
rangess = [ranges_memr, ranges_anr, ranges_skew, ranges_regularity]
# rangess = [rangess[cnn]]
# cnttt = 0

for cnttt in range(len(features)):
    feat, ranges_feat = features[cnttt], rangess[cnttt]
    print(feat)
    group_data_system_list=[]
    for system in ranges_dev:
        print('\n'+system)
        formats = [x.split("\n")[1] for x in ranges_impl_arch_backup if 
                   ((x.split(" )")[0].split("( ")[1] == system) and 
                    (x.split("\n")[1] in set(group_data[group_data['System']==x.split(" )")[0].split("( ")[1]]['implementation'])))]
        
        if(research_only):
            formats = [ i if i in research_formats else '' for i in formats]
            while('' in formats):
                formats.remove('')

        group_data_sys = group_data[group_data['System']==system]

        new_df_list = []
        for f in formats:
            # print(system, '\t', f, '\t', group_data_sys_f.shape[0])
            # feat, ranges_feat = features[cnttt], rangess[cnttt]
            ndf_row = print_distribution_row100(feat, feat, ranges_feat, group_data_sys, verbose=False, format_name = f)
            new_df_list.append(ndf_row)

        new_df = pd.DataFrame(new_df_list, columns = ['Format'] + ranges_feat)
        if(new_df.shape[0]>0):
            new_df.set_index('Format', inplace=True)
            new_df = new_df.T
            display(new_df)


mem_range

Tesla-V100


Format,cu-COO,cu-CSR,cu-HYB,CSR5
[4-8],66.67% (38.89 GF),17.03% (40.42 GF),0.0% (nan GF),16.31% (34.08 GF)
[8-16],49.5% (48.94 GF),29.09% (56.99 GF),0.0% (nan GF),21.4% (53.15 GF)
[16-32],10.18% (63.29 GF),34.65% (76.43 GF),8.82% (58.91 GF),46.35% (69.56 GF)
[32-64],6.26% (69.13 GF),4.23% (53.86 GF),31.3% (71.04 GF),58.21% (79.4 GF)
[64-128],1.3% (55.37 GF),46.96% (89.84 GF),0.0% (nan GF),51.74% (106.12 GF)
[128-256],0.19% (60.11 GF),16.73% (85.07 GF),42.08% (98.96 GF),41.01% (110.37 GF)
[256-512],0.79% (57.77 GF),23.1% (67.83 GF),45.24% (108.1 GF),30.87% (119.75 GF)
[512-1024],1.38% (63.1 GF),23.11% (49.39 GF),57.11% (113.98 GF),18.41% (120.84 GF)
[1024-2048],4.04% (56.85 GF),25.48% (46.93 GF),52.67% (111.08 GF),17.81% (121.58 GF)



Tesla-A100


Format,cu-COO,cu-CSR,SELL-C-s
[4-8],83.52% (40.15 GF),0.0% (nan GF),0.0% (nan GF)
[8-16],94.37% (86.09 GF),0.0% (nan GF),5.63% (77.29 GF)
[16-32],87.44% (113.82 GF),10.51% (122.6 GF),0.55% (103.08 GF)
[32-64],100.0% (122.69 GF),0.0% (nan GF),0.0% (nan GF)
[64-128],15.93% (125.96 GF),84.07% (150.58 GF),0.0% (nan GF)
[128-256],9.37% (125.33 GF),90.26% (158.49 GF),0.0% (nan GF)
[256-512],8.64% (127.79 GF),88.2% (162.81 GF),0.39% (129.13 GF)
[512-1024],12.13% (131.81 GF),82.51% (161.82 GF),1.5% (141.15 GF)
[1024-2048],10.37% (135.46 GF),80.54% (159.07 GF),3.98% (155.81 GF)



Grace-H100


...

Format,Naive-CSR,Vec-CSR,MKL-IE,CSR5,SparseX,Merge-CSR,SELL-C-s
SS,1.45% (7.21 GF),7.72% (31.82 GF),22.24% (30.53 GF),0.05% (22.72 GF),28.67% (12.72 GF),15.11% (8.17 GF),24.76% (8.33 GF)
SM,0.99% (7.38 GF),8.83% (29.0 GF),36.13% (18.15 GF),0.16% (21.47 GF),12.17% (12.79 GF),16.17% (9.38 GF),25.55% (8.53 GF)
SL,0.56% (7.66 GF),8.04% (31.56 GF),31.31% (25.77 GF),0.28% (21.91 GF),29.72% (12.94 GF),11.21% (9.57 GF),18.88% (7.82 GF)
MS,0.83% (10.59 GF),7.06% (33.81 GF),26.27% (27.86 GF),0.26% (10.56 GF),25.44% (12.68 GF),15.63% (9.02 GF),24.51% (8.93 GF)
MM,0.95% (10.85 GF),9.08% (29.35 GF),41.7% (18.81 GF),0.2% (22.89 GF),7.23% (12.55 GF),16.71% (9.53 GF),24.13% (8.94 GF)
ML,0.36% (8.44 GF),7.73% (32.79 GF),36.84% (27.93 GF),0.18% (16.25 GF),29.2% (13.32 GF),8.54% (9.62 GF),17.16% (8.1 GF)
LS,0.72% (8.91 GF),7.61% (35.27 GF),23.15% (32.66 GF),0.1% (24.55 GF),36.27% (13.19 GF),14.87% (8.77 GF),17.28% (8.92 GF)
LM,0.38% (8.43 GF),8.35% (35.07 GF),34.74% (32.44 GF),0.05% (25.19 GF),31.18% (13.24 GF),11.02% (9.48 GF),14.3% (8.8 GF)
LL,0.22% (10.16 GF),13.99% (48.69 GF),45.87% (42.34 GF),0.07% (31.51 GF),33.12% (14.64 GF),2.9% (22.88 GF),3.84% (9.1 GF)



IBM-POWER9


Format,Naive-CSR,Bal-CSR,SparseX,Merge-CSR
SS,5.52% (4.23 GF),11.52% (23.29 GF),27.38% (11.47 GF),55.57% (9.52 GF)
SM,5.98% (20.21 GF),16.01% (26.18 GF),5.48% (13.06 GF),72.53% (11.61 GF)
SL,4.86% (30.49 GF),18.13% (32.98 GF),0.0% (nan GF),77.01% (16.53 GF)
MS,6.02% (10.97 GF),17.91% (25.6 GF),16.25% (12.29 GF),59.81% (12.39 GF)
MM,5.07% (29.75 GF),17.61% (30.59 GF),0.45% (13.26 GF),76.87% (14.27 GF)
ML,6.56% (39.22 GF),24.08% (35.59 GF),0.0% (nan GF),69.36% (18.19 GF)
LS,4.06% (28.1 GF),14.35% (29.52 GF),24.43% (21.97 GF),57.15% (16.02 GF)
LM,7.46% (35.52 GF),23.64% (34.54 GF),10.07% (22.32 GF),58.83% (17.98 GF)
LL,14.54% (42.79 GF),46.96% (35.9 GF),2.46% (19.72 GF),36.03% (19.74 GF)



Alveo-U280


Format,Xilinx-lib
SS,100.0% (20.4 GF)
SM,100.0% (17.2 GF)
SL,100.0% (8.7 GF)
MS,100.0% (20.6 GF)
MM,100.0% (18.09 GF)
ML,100.0% (11.9 GF)
LS,100.0% (20.5 GF)
LM,100.0% (19.23 GF)
LL,100.0% (16.48 GF)


# performance median (and percentage of matrices per format) per feature range
# (col100 - bad)

In [55]:
def print_distribution_col100(feature, feat_categ, ranges_feature, data, verbose = True, format_name=''):
    total = data.shape[0]
    if(verbose):
        print(feature + ' distribution in dataset')
    ndf_row = [format_name + '('+str(total)+')']
    for val in ranges_feature:
        data_feat = data[data[feat_categ]==val]
        cnt = data_feat.shape[0]
        new_string = str(np.round(100*cnt/total,2)) + '% ('+ str(np.round(data_feat['gflops'].median(),2)) + ' GF)'
        if(verbose):
            print(val, '\t', cnt,'/', total,'\t', new_string)
        ndf_row.append(new_string)
    return ndf_row

group_data = group_system_best
# group_data = groupreps

# ranges_dev = ['Tesla-A100', 'Tesla-V100', 'AMD-EPYC', 'ARM-NEON', 'Alveo-U280']
# ranges_dev = ['Tesla-A100', 'AMD-EPYC1', 'INTEL-GOLD2', 'ARM-NEON', 'Alveo-U280']
ranges_dev = ['Tesla-V100', 'Tesla-A100', 'Grace-H100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
research_formats = ['Merge-CSR', 'SparseX', 'SELL-C-s', 'CSR5']

research_only = False
# research_only = True

features = ['mem_range', 'anr_categ', 'skew_categ', 'regularity']
rangess = [ranges_memr, ranges_anr, ranges_skew, ranges_regularity]
cnttt = 0
        
group_data_system_list=[]
for system in ranges_dev:
    print('---------------------')
    print(system)
    # formats = set(group_data[(group_data['System']==system)]['implementation'])
    formats = [x.split("\n")[1] for x in ranges_impl_arch_backup if 
               ((x.split(" )")[0].split("( ")[1] == system) and 
                (x.split("\n")[1] in set(group_data[group_data['System']==x.split(" )")[0].split("( ")[1]]['implementation'])))]

    if(research_only):
        formats = [ i if i in research_formats else '' for i in formats]
        while('' in formats):
            formats.remove('')

    group_data_sys = group_data[group_data['System']==system]

    new_df_list = []
    for f in formats:
        group_data_sys_f = group_data_sys[group_data_sys['implementation'] == f]
        # print(system, '\t', f, '\t', group_data_sys_f.shape[0])
        feat, ranges_feat = features[cnttt], rangess[cnttt]
        ndf_row = print_distribution_col100(feat, feat, ranges_feat, group_data_sys_f, verbose=False, format_name = f)
        new_df_list.append(ndf_row)

    new_df = pd.DataFrame(new_df_list, columns = ['Format'] + ranges_feat)
    new_df.set_index('Format', inplace=True)
    new_df = new_df.T
    display(new_df)


---------------------
Tesla-V100


Format,cu-COO(1396),cu-CSR(3300),cu-HYB(4284),CSR5(3712)
[4-8],19.91% (38.89 GF),2.15% (40.42 GF),0.0% (nan GF),1.83% (34.08 GF)
[8-16],53.51% (48.94 GF),13.3% (56.99 GF),0.0% (nan GF),8.7% (53.15 GF)
[16-32],12.89% (63.29 GF),18.58% (76.43 GF),3.64% (58.91 GF),22.09% (69.56 GF)
[32-64],2.65% (69.13 GF),0.76% (53.86 GF),4.32% (71.04 GF),9.27% (79.4 GF)
[64-128],0.21% (55.37 GF),3.27% (89.84 GF),0.0% (nan GF),3.21% (106.12 GF)
[128-256],0.21% (60.11 GF),8.06% (85.07 GF),15.62% (98.96 GF),17.56% (110.37 GF)
[256-512],1.07% (57.77 GF),13.24% (67.83 GF),19.98% (108.1 GF),15.73% (119.75 GF)
[512-1024],1.72% (63.1 GF),12.21% (49.39 GF),23.25% (113.98 GF),8.65% (120.84 GF)
[1024-2048],7.81% (56.85 GF),20.85% (46.93 GF),33.19% (111.08 GF),12.96% (121.58 GF)


---------------------
Tesla-A100


Format,cu-COO(6067),cu-CSR(8516),SELL-C-s(313)
[4-8],7.43% (40.15 GF),0.0% (nan GF),0.0% (nan GF)
[8-16],28.75% (86.09 GF),0.0% (nan GF),33.23% (77.29 GF)
[16-32],31.68% (113.82 GF),2.71% (122.6 GF),3.83% (103.08 GF)
[32-64],11.7% (122.69 GF),0.0% (nan GF),0.0% (nan GF)
[64-128],0.71% (125.96 GF),2.67% (150.58 GF),0.0% (nan GF)
[128-256],2.92% (125.33 GF),20.03% (158.49 GF),0.0% (nan GF)
[256-512],3.25% (127.79 GF),23.61% (162.81 GF),2.88% (129.13 GF)
[512-1024],4.14% (131.81 GF),20.06% (161.82 GF),9.9% (141.15 GF)
[1024-2048],5.59% (135.46 GF),30.92% (159.07 GF),41.53% (155.81 GF)


---------------------
Grace-H100


...

Format,Naive-CSR(115),Vec-CSR(1313),MKL-IE(4980),CSR5(22),SparseX(3861),Merge-CSR(1990),SELL-C-s(2968)
[4-8],0.0% (nan GF),18.43% (37.43 GF),5.18% (44.83 GF),0.0% (nan GF),0.0% (nan GF),2.01% (36.22 GF),0.0% (nan GF)
[8-16],3.48% (16.49 GF),30.16% (42.8 GF),25.56% (53.18 GF),4.55% (33.55 GF),0.0% (nan GF),6.18% (32.58 GF),1.25% (25.73 GF)
[16-32],9.57% (20.57 GF),25.06% (28.92 GF),26.85% (32.29 GF),54.55% (23.43 GF),0.0% (nan GF),23.37% (21.4 GF),0.37% (28.21 GF)
[32-64],2.61% (11.02 GF),7.16% (23.43 GF),8.05% (26.33 GF),13.64% (21.47 GF),0.0% (nan GF),8.19% (14.34 GF),1.55% (15.94 GF)
[64-128],17.39% (7.84 GF),0.99% (12.46 GF),1.18% (10.55 GF),0.0% (nan GF),2.46% (13.95 GF),4.17% (8.08 GF),0.0% (nan GF)
[128-256],21.74% (7.82 GF),2.97% (11.98 GF),5.1% (11.13 GF),22.73% (10.47 GF),22.46% (13.85 GF),14.42% (8.57 GF),13.92% (9.13 GF)
[256-512],27.83% (8.21 GF),1.9% (11.78 GF),7.99% (9.64 GF),4.55% (10.56 GF),24.61% (13.07 GF),15.98% (8.13 GF),17.62% (9.16 GF)
[512-1024],9.57% (7.82 GF),1.83% (11.39 GF),6.57% (9.11 GF),0.0% (nan GF),19.99% (12.91 GF),13.37% (8.02 GF),22.57% (8.73 GF)
[1024-2048],7.83% (6.62 GF),3.2% (4.88 GF),9.96% (8.98 GF),0.0% (nan GF),30.48% (12.64 GF),12.31% (7.75 GF),42.69% (7.61 GF)


---------------------
IBM-POWER9


Format,Naive-CSR(994),Bal-CSR(3100),SparseX(1658),Merge-CSR(9511)
[4-8],7.04% (30.96 GF),6.42% (29.0 GF),0.0% (nan GF),2.85% (20.1 GF)
[8-16],26.46% (33.42 GF),23.13% (32.56 GF),0.0% (nan GF),9.13% (20.93 GF)
[16-32],30.18% (34.25 GF),27.84% (33.32 GF),0.0% (nan GF),10.54% (20.74 GF)
[32-64],2.01% (36.71 GF),9.52% (31.87 GF),0.0% (nan GF),4.15% (21.08 GF)
[64-128],1.01% (30.98 GF),1.03% (33.12 GF),2.23% (16.16 GF),2.01% (13.15 GF)
[128-256],0.7% (30.82 GF),6.52% (26.67 GF),17.73% (15.64 GF),14.58% (14.25 GF)
[256-512],2.72% (3.75 GF),5.29% (23.87 GF),22.56% (13.44 GF),17.68% (13.29 GF)
[512-1024],4.02% (3.7 GF),5.48% (15.11 GF),20.14% (16.22 GF),16.04% (12.11 GF)
[1024-2048],12.37% (4.35 GF),11.1% (12.1 GF),37.33% (15.68 GF),22.61% (10.58 GF)


---------------------
Alveo-U280


Format,Xilinx-lib(10391)
[4-8],5.2% (16.9 GF)
[8-16],17.75% (19.57 GF)
[16-32],20.77% (19.6 GF)
[32-64],6.83% (15.6 GF)
[64-128],2.58% (14.9 GF)
[128-256],16.55% (16.2 GF)
[256-512],13.6% (19.3 GF)
[512-1024],7.58% (21.2 GF)
[1024-2048],6.42% (22.8 GF)


---
# distribution of each feature range for each device

In [56]:
group_data = group_system_best

# what_list = ['all', 'small', 'big']
what_list = ['small', 'big']

# fpga_thres = 64 # fpga_thres = 128 # fpga_thres = 256 # fpga_thres = 512
fpga_thres = 256
ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'Grace-H100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'ARM-GRACE', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']
features = ['mem_range', 'anr_categ', 'skew_categ', 'regularity']
rangess = [ranges_memr, ranges_anr, ranges_skew, ranges_regularity]
cnttt = 0

group_data_system_list=[]
for system in ranges_dev:
    print('-------------------------------------------------------------')
    print('---', system, '---')
    print(group_data[group_data['System']==system].shape[0], 'matrices')
    select_str_small, tmp = select_region3(system, 'small', group_data, fpga_thres = fpga_thres )
    select_str_big, tmp = select_region3(system, 'big', group_data, fpga_thres = fpga_thres )
    select_str_all, tmp = select_region3(system, 'all', group_data, fpga_thres = fpga_thres )
    group_data_small_system = group_data[select_str_small]
    group_data_small_system['mark_perf'] = 'small'
    group_data_big_system = group_data[select_str_big]
    group_data_big_system['mark_perf'] = 'big'
    # group_data_all_system = group_data[select_str_all]
    # group_data_all_system['mark_perf'] = 'all'
    if(len(what_list)==2):
        group_data_system = pd.concat([group_data_small_system, group_data_big_system])
    else:
        group_data_system = pd.concat([group_data_small_system, group_data_big_system, group_data_all_system])

    group_data_good_system_custom_small = group_data_small_system[group_data_small_system['System']==system]
    print('small :', group_data_small_system.shape[0], '\tbig :',group_data_big_system.shape[0],'\n')
    group_data_good_system_custom_big = group_data_big_system[group_data_big_system['System']==system]

    feat = features[cnttt]
    print(feat)
    ranges_feat = rangess[cnttt]
    for feat_val in ranges_feat:
        group_systemaki = group_data_good_system_custom_small[group_data_good_system_custom_small[feat]==feat_val]
        if(group_systemaki.shape[0]>0):
            print('small', feat_val, '\t', round(group_systemaki['gflops'].median(),2),'GFLOPs\t', '(', group_systemaki.shape[0],' matrices)')
    print()
    for feat_val in ranges_feat:
        group_systemaki = group_data_good_system_custom_big[group_data_good_system_custom_big[feat]==feat_val]
        if(group_systemaki.shape[0]>0):
            print('big  ', feat_val, '\t', round(group_systemaki['gflops'].median(),2),'GFLOPs\t', '(', group_systemaki.shape[0],' matrices)')


    # plot_dev_perf3(group_data_system, system)
    group_data_system_list.append(group_data_system) 
group_data_system_list = pd.concat(group_data_system_list)

# features = ['avg_nz_row', 'skew_coeff', 'regularity']
# features = ['avg_nz_row'] # features = ['skew_coeff'] # features = [ 'regularity']
# for feat in features:
#     plot_dev_perf(feat, 10, 'all', group_data_system_list)

-------------------------------------------------------------
--- Tesla-P100 ---
9918 matrices
small : 2926 	big : 6480 

mem_range
small [8-16] 	 36.32 GFLOPs	 ( 1128  matrices)
small [16-32] 	 44.38 GFLOPs	 ( 1358  matrices)
small [32-64] 	 43.92 GFLOPs	 ( 440  matrices)

big   [64-128] 	 47.54 GFLOPs	 ( 180  matrices)
big   [128-256] 	 52.13 GFLOPs	 ( 1260  matrices)
big   [256-512] 	 51.43 GFLOPs	 ( 1530  matrices)
big   [512-1024] 	 49.96 GFLOPs	 ( 1350  matrices)
big   [1024-2048] 	 46.43 GFLOPs	 ( 2160  matrices)
-------------------------------------------------------------
--- Tesla-V100 ---
12692 matrices
small : 3869 	big : 8156 

mem_range
small [8-16] 	 51.7 GFLOPs	 ( 1509  matrices)
small [16-32] 	 69.75 GFLOPs	 ( 1769  matrices)
small [32-64] 	 75.35 GFLOPs	 ( 591  matrices)

big   [64-128] 	 99.69 GFLOPs	 ( 230  matrices)
big   [128-256] 	 103.69 GFLOPs	 ( 1590  matrices)
big   [256-512] 	 109.78 GFLOPs	 ( 1892  matrices)
big   [512-1024] 	 108.04 GFLOPs	 ( 1744  matrice

---
# plot ANGELS vs DEVILS

In [ ]:
def select_region2(system, what, group_data):
    # plot synthetic dataset only
    select_criterion = (group_data['Arch'] != '')
    if(system!=''):
        select_criterion = select_criterion & (group_data['System']==system)
    select_str = ''

    # features = ['A_mem_footprint','avg_nz_row', 'skew_coeff', 'avg_num_neighbours', 'cross_row_similarity']
    features = ['A_mem_footprint','avg_nz_row', 'skew_coeff', 'regularity']
    for feat in features:
        if(feat=='regularity'):
            if(what == 'good'):
                irr_val_list = ['LL']
                if(system=='Tesla-V100'):
                    irr_val_list = ['LL']
                if(system=='Tesla-A100'):
                    irr_val_list = ['LL']
                if(system=='AMD-EPYC-64'):
                    irr_val_list = ['LM','LL']
                if(system=='AMD-EPYC-24'):
                    irr_val_list = ['LM','LL']
                if(system=='ARM-NEON'):
                    irr_val_list = ['LL']
                if(system=='INTEL-XEON'):
                    irr_val_list = ['LM','LL']
                if(system=='IBM-POWER9'):
                    irr_val_list = ['LL']
                if(system=='Alveo-U280'):
                    irr_val_list = ['SS', 'MS', 'LS']
            elif(what == 'bad'):
                irr_val_list = ['SS']
                if(system=='Tesla-V100'):
                    irr_val_list = ['SS', 'SM']
                if(system=='Tesla-A100'):
                    irr_val_list = ['SS']
                if(system=='AMD-EPYC-64'):
                    irr_val_list = ['SS', 'SM', 'MS']
                if(system=='AMD-EPYC-24'):
                    irr_val_list = ['SS', 'SM', 'MS']
                if(system=='ARM-NEON'):
                    irr_val_list = ['SS', 'SM', 'SL', 'MS', 'LS']
                if(system=='INTEL-XEON'):
                    irr_val_list = ['SS', 'SM', 'MS']
                if(system=='IBM-POWER9'):
                    irr_val_list = ['SS', 'SM', 'SL', 'MS', 'LS']
                if(system=='Alveo-U280'):
                    irr_val_list = ['SL','ML','LL']
            elif(what == 'all'):
                irr_val_list = ['SS', 'SM', 'SL', 'MS', 'MM', 'ML', 'LS', 'LM', 'LL']

            select_str = select_str + feat + '=['+ ','.join(irr_val_list) +']' + '  '
            for irr_val in irr_val_list:
                if(irr_val == irr_val_list[0]):
                    select_criterion_irr = (group_data[feat] == irr_val)
                else:
                    select_criterion_irr = select_criterion_irr | (group_data[feat] == irr_val)
            select_criterion = select_criterion & select_criterion_irr
        else:
            if(what == 'good'):
                if(feat == 'A_mem_footprint'):
                    if(system=='Tesla-V100'):
                        min_feat, max_feat = 256, 1024
                    if(system=='Tesla-A100'):
                        min_feat, max_feat = 256, 1024
                    if(system=='AMD-EPYC-64'):
                        min_feat, max_feat = 64, 256
                    if(system=='AMD-EPYC-24'):
                        min_feat, max_feat = 32, 128
                    if(system=='ARM-NEON'):
                        min_feat, max_feat = 4, 64
                    if(system=='INTEL-XEON'):
                        min_feat, max_feat = 4, 32
                    if(system=='IBM-POWER9'):
                        min_feat, max_feat = 4, 64
                    if(system=='Alveo-U280'):
                        min_feat, max_feat = 4, 256
                if(feat == 'avg_nz_row'):
                    if(system=='Tesla-V100'):
                        min_feat, max_feat = 50, 510
                    if(system=='Tesla-A100'):
                        min_feat, max_feat = 50, 510
                    if(system=='AMD-EPYC-64'):
                        min_feat, max_feat = 100, 510
                    if(system=='AMD-EPYC-24'):
                        min_feat, max_feat = 100, 510
                    if(system=='ARM-NEON'):
                        min_feat, max_feat = 10, 510
                    if(system=='INTEL-XEON'):
                        min_feat, max_feat = 100, 510
                    if(system=='IBM-POWER9'):
                        min_feat, max_feat = 10, 510
                    if(system=='Alveo-U280'):
                        min_feat, max_feat = 100, 510
                if(feat == 'skew_coeff'):
                    if(system=='Tesla-V100'):
                        min_feat, max_feat = 0, 1000
                    if(system=='Tesla-A100'):
                        min_feat, max_feat = 0, 1000
                    if(system=='AMD-EPYC-64'):
                        min_feat, max_feat = 0, 100
                    if(system=='AMD-EPYC-24'):
                        min_feat, max_feat = 0, 100
                    if(system=='ARM-NEON'):
                        min_feat, max_feat = 10, 15
                    if(system=='INTEL-XEON'):
                        min_feat, max_feat = 0, 100
                    if(system=='IBM-POWER9'):
                        min_feat, max_feat = 10, 15
                    if(system=='Alveo-U280'):
                        min_feat, max_feat = 0, 100

            if(what == 'bad'):
                if(feat == 'A_mem_footprint'):
                    if(system=='Tesla-V100'):
                        min_feat, max_feat = 1024, 2048
                    if(system=='Tesla-A100'):
                        min_feat, max_feat = 1024, 2048
                    if(system=='AMD-EPYC-64'):
                        min_feat, max_feat = 256, 2048
                    if(system=='AMD-EPYC-24'):
                        min_feat, max_feat = 128, 2048
                    if(system=='ARM-NEON'):
                        min_feat, max_feat = 64, 2048
                    if(system=='INTEL-XEON'):
                        min_feat, max_feat = 32, 2048
                    if(system=='IBM-POWER9'):
                        min_feat, max_feat = 64, 2048
                    if(system=='Alveo-U280'):
                        min_feat, max_feat = 256, 2048
                if(feat == 'avg_nz_row'):
                    if(system=='Tesla-V100'):
                        min_feat, max_feat = 0, 10
                    if(system=='Tesla-A100'):
                        min_feat, max_feat = 0, 6
                    if(system=='AMD-EPYC-64'):
                        min_feat, max_feat = 0, 20
                    if(system=='AMD-EPYC-24'):
                        min_feat, max_feat = 0, 20
                    if(system=='ARM-NEON'):
                        min_feat, max_feat = 0, 10
                    if(system=='INTEL-XEON'):
                        min_feat, max_feat = 0, 20
                    if(system=='IBM-POWER9'):
                        min_feat, max_feat = 0, 10
                    if(system=='Alveo-U280'):
                        min_feat, max_feat = 0, 10
                if(feat == 'skew_coeff'):
                    if(system=='Tesla-V100'):
                        min_feat, max_feat = 1000, 180000
                    if(system=='Tesla-A100'):
                        min_feat, max_feat = 1000, 180000
                    if(system=='AMD-EPYC-64'):
                        min_feat, max_feat = 1000, 180000
                    if(system=='AMD-EPYC-24'):
                        min_feat, max_feat = 1000, 180000
                    if(system=='ARM-NEON'):
                        min_feat, max_feat = 20, 180000
                    if(system=='INTEL-XEON'):
                        min_feat, max_feat = 1000, 180000
                    if(system=='IBM-POWER9'):
                        min_feat, max_feat = 20, 180000
                    if(system=='Alveo-U280'):
                        min_feat, max_feat = 1000, 180000

            if(what == 'all'):
                if(feat == 'A_mem_footprint'):
                    min_feat, max_feat = 0, 2048
                if(feat == 'avg_nz_row'):
                    min_feat, max_feat = 0, 510
                if(feat == 'skew_coeff'):
                    min_feat, max_feat = 0, 180000
            select_str = select_str + feat + '=['+str(min_feat)+'-'+str(max_feat)+']' + ' '
            select_criterion = select_criterion & (group_data[feat] >= min_feat) & (group_data[feat] <= max_feat)
    return select_criterion, select_str

def plot_dev_perf3(group_data):
    if(len(what_list)==2):
        system2 = 'Feature Impact-ALL_DEVICES_bad_good'
    else:
        system2 = 'Feature Impact-ALL_DEVICES_bad_all_good'
    plot_x_axis = ['System']
    plot_y_axis = ['gflops']#,'energy_efficiency']
    plot_id = system2
    select_criterion = (group_data['Arch'] != '')

    group_system_select = group_data[select_criterion]
    select_str = 'All features intuitively fixed'
    harry_plotter('box', group_system_select, plot_x_axis, plot_y_axis, select_str, plot_id, hue_hue = 'mark_perf', hue_hue_order = what_list, no_title=True,
                 rotation=30)#, show_size=True)

In [ ]:
###################################################################################################################
###################################################################################################################

# par=0.85
# par2=0.9
# plt.rc('figure', figsize=(1.5*width*par2,1.5*height*par))
plt.rc('figure', figsize=(width,height))

group_data = group_system_best

ranges_dev = ['Tesla-V100', 'Tesla-A100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'INTEL-XEON', 'IBM-POWER9', 'Alveo-U280']

# what_list = ['bad', 'good']
what_list = ['Devils', 'Angels']

# what_list = ['bad', 'all', 'good']

group_data_system_list=[]
for system in ranges_dev:
    # print('--', system, '--')
    select_str_all, tmp1 = select_region2(system, 'all', group_data)
    select_str_bot, tmp2 = select_region2(system, 'bad', group_data)
    select_str_good, tmp3 = select_region2(system, 'good', group_data)

    # print(tmp1)
    # print(tmp2)

    group_data_all_system = group_data[select_str_all]
    group_data_all_system['mark_perf'] = 'all'
    group_data_bot_system = group_data[select_str_bot]
    group_data_bot_system['mark_perf'] = 'Devils'

    if(system=='Tesla-A100'): # !!!!!!
        group_data_bot_system = group_data_bot_system[group_data_bot_system['gflops']<80]
        
    group_data_good_system = group_data[select_str_good]
    group_data_good_system['mark_perf'] = 'Angels'
    
    # group_data_system = pd.concat([group_data_all_system, group_data_bot_system, group_data_good_system])
    group_data_system = pd.concat([group_data_bot_system, group_data_good_system])

    # print(system, '\t', 'bad', group_data_bot_system.shape[0], '\tgood', group_data_good_system.shape[0], )
    # print('bad ', group_data_bot_system['gflops'].min(), group_data_bot_system['gflops'].max())
    # print('good', group_data_good_system['gflops'].min(), group_data_good_system['gflops'].max())
    group_data_system_list.append(group_data_system)
group_data_system_all = pd.concat(group_data_system_list)

cp3_special = [(0.9727323000000001, 0.7353159, 0.5297757000000001), (0.850771241830065, 0.21914033064206087, 0.18984390618992708), (0.23574163783160318, 0.597845444059977, 0.33886505190311433)]
cp2_special = [(0.850771241830065, 0.21914033064206087, 0.18984390618992708), (0.23574163783160318, 0.597845444059977, 0.33886505190311433)]

if(len(what_list)==2):
    sns.set_palette(cp2_special)
else:
    sns.set_palette(cp3_special)
plot_dev_perf3(group_data_system_all)


---
# plot RESEARCH vs VENDOR formats
# Research only (keep best of of Research formats)

In [ ]:
%%time
groupreps1 = groupreps
discard_list = ['cuSPARSE-COO', 'cuSPARSE-CSR', 'cuSPARSE-HYB',
                'MKL-IE', 'AOCL', 'ARM-library', 'Xilinx-library',
                'Vectorized-CSR', 'Naive-CSR', 'Balanced-CSR'
               ]
for impl in discard_list:
    groupreps1 = groupreps1[groupreps1.implementation != impl]

group_system1 = groupreps1.groupby(['mtx_name','distribution','placement','seed',
                                  'm','n','nz','density','A_mem_footprint','mem_range',
                                  'avg_nz_row','std_nz_row',
                                  'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                                  'avg_scattering','std_scattering','avg_scattering_scaled','std_scattering_scaled',
                                  'skew_coeff','avg_num_neighbours','cross_row_similarity',
                                  'System','Arch'], as_index = False, observed = True)
reslist = []
for desc, experiment in group_system1:
    best_format = experiment['implementation'].iloc[experiment['gflops'].argmax()]
    outrow = experiment[experiment['implementation'] == best_format]
    # if(len(outrow)>1):
    #     print(len(outrow), outrow)
    reslist.append(outrow.values.tolist()[0])             

group_system_research_best = pd.DataFrame(reslist, columns = header_names+extra_header_names)#+['lib_type'])
print(group_system_research_best.shape)
for sys in set(group_system_research_best['System']):
    print(sys, '\t', group_system_research_best[group_system_research_best['System']==sys].shape[0])

# Vendors only (keep best of of Vendors formats)

In [ ]:
%%time
groupreps2 = groupreps
discard_list= ['CSR5', 'SELL-32-1', 'SparseX', 'Merge-CSR']
for impl in discard_list:
    groupreps2 = groupreps2[groupreps2.implementation != impl]

group_system2 = groupreps2.groupby(['mtx_name','distribution','placement','seed',
                                  'm','n','nz','density','A_mem_footprint','mem_range',
                                  'avg_nz_row','std_nz_row',
                                  'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                                  'avg_scattering','std_scattering','avg_scattering_scaled','std_scattering_scaled',
                                  'skew_coeff','avg_num_neighbours','cross_row_similarity',
                                  'System','Arch'], as_index = False, observed = True)
reslist = []
for desc, experiment in group_system2:
    best_format = experiment['implementation'].iloc[experiment['gflops'].argmax()]
    outrow = experiment[experiment['implementation'] == best_format]
    # if(len(outrow)>1):
    #     print(len(outrow), outrow)
    reslist.append(outrow.values.tolist()[0])             

group_system_vendors_best = pd.DataFrame(reslist, columns = header_names+extra_header_names)#+['lib_type'])
print(group_system_vendors_best.shape)
for sys in set(group_system_vendors_best['System']):
    print(sys, '\t', group_system_vendors_best[group_system_vendors_best['System']==sys].shape[0])

In [ ]:
# to verify that each boxplot will contain same amount of measurements (slightly different maybe)
for sys in set(group_system_best['System']):
    print(sys, '\t', group_system_best[group_system_best['System']==sys].shape[0])

# Now plot All vs Research vs Vendors (keep best of for each matrix (only within Research and Vendors though...))

In [ ]:
def plot_dev_perf3(group_data):
    system2 = 'Format Impact-ALL_DEVICES_research_vendor'
    plot_x_axis = ['System']
    plot_y_axis = ['gflops']#,'energy_efficiency']
    plot_id = system2

    group_system_select = group_data
    select_str = ''
    print(select_str)
    harry_plotter('box', group_system_select, plot_x_axis, plot_y_axis, select_str, plot_id, hue_hue = 'mark_perf', hue_hue_order = what_list, no_title=True)#, show_size=True)

###################################################################################################################
###################################################################################################################

group_data = group_system_best

# group_data['lib_type'] = 'Vendor'     
# for f in set(group_data['implementation']):
#     if f in ['Merge-CSR', 'SparseX', 'SELL-C-s', 'CSR5']: # decide whether Naive-CSR Vectorized-CSR will be included
#     # if f in ['Merge-CSR', 'SparseX', 'SELL-C-s', 'CSR5', 'Naive-CSR', 'Vectorized-CSR']:
#         group_data.loc[group_data['implementation']==f,'lib_type'] = 'Research' 

group_system_research_best['lib_type'] = 'Research'
group_system_vendors_best['lib_type'] = 'Vendor'
ranges_dev = ['Tesla-P100', 'Tesla-V100', 'Tesla-A100', 'AMD-EPYC-24', 'AMD-EPYC-64', 'ARM-NEON', 'INTEL-XEON', 'INTEL-ICY', 'IBM-POWER9', 'Alveo-U280']

what_list = ['All', 'Research', 'Vendor']
# what_list = ['All', 'Vendor']
# what_list = ['Vendor', 'All']

group_data_system_list=[]
for system in ranges_dev:
    print('--', system, '--')
    group_data_system = group_data[group_data['System'] == system]

    group_data_all_system = group_data_system
    group_data_all_system['mark_perf'] = 'All'

    # group_data_res_system = group_data_system[group_data_system['lib_type'] == 'Research']
    group_data_res_system = group_system_research_best[group_system_research_best['System'] == system] # group_data_system[group_data_system['lib_type'] == 'Research']
    group_data_res_system['mark_perf'] = 'Research'

    # group_data_ven_system = group_data_system[group_data_system['lib_type'] == 'Vendor']
    group_data_ven_system = group_system_vendors_best[group_system_vendors_best['System'] == system] # group_data_system[group_data_system['lib_type'] == 'Vendor']
    group_data_ven_system['mark_perf'] = 'Vendor'

    # group_data_system = pd.concat([group_data_res_system, group_data_ven_system])
    group_data_system = pd.concat([group_data_all_system, group_data_res_system, group_data_ven_system])

    print(system, '\t', 'All', group_data_all_system.shape[0], 'Research', group_data_res_system.shape[0], '\tVendor', group_data_ven_system.shape[0], )
    print('All ', round(group_data_all_system['gflops'].min(),2), '-', round(group_data_all_system['gflops'].max(),2))
    print('Research ', round(group_data_res_system['gflops'].min(),2), '-', round(group_data_res_system['gflops'].max(),2))
    print('Vendor   ', round(group_data_ven_system['gflops'].min(),2), '-', round(group_data_ven_system['gflops'].max(),2))
    group_data_system_list.append(group_data_system)

group_data_system_all = pd.concat(group_data_system_list)

if(len(what_list)==2):
    sns.set_palette(cp2_special)
else:
    sns.set_palette(cp3_special)

plt.rc('figure', figsize=(width,height))
plot_dev_perf3(group_data_system_all)


---
---
# find which matrices of large size (256-2048 MB) perform bad on GPU
### keep only those under 50 GFLOPs
### It's only 1% of the dataset, and its features are
    rows : 5M - 28.5M
    avg nnz/row : 5, 10
    regularity : MS, SM, SS (small regularity)

In [57]:
def find_bad_features_system(system, thres):
    group_data = group_system_best[group_system_best['System']==system]
    group_data = group_data[(group_data['mem_range'] == '[256-512]') | 
                            (group_data['mem_range'] == '[512-1024]') | 
                            (group_data['mem_range'] == '[1024-2048]')]
    group_data = group_data[group_data['gflops'] < thres]

    print(group_data.shape)

    group_data = group_data.round({'avg_nz_row':0, 'skew_coeff': -1})
    print('rows : ', group_data['m'].min().round(-4), '-', group_data['m'].max().round(-4))
    print('avg_nz_row : ', set(group_data['avg_nz_row']))
    print('skew_coeff : ', set(group_data['skew_coeff']))
    print('regularity : ', set(group_data['regularity']))

    width_hist = 10
    height_hist = 5

    feature = 'm'
    hist = group_data.hist(column=feature, bins=20, figsize=(width_hist, height_hist))

    feature = 'avg_nz_row'
    hist = group_data.hist(column=feature, bins=20, figsize=(width_hist, height_hist))

    feature = 'skew_coeff'
    hist = group_data.hist(column=feature, bins=20, figsize=(width_hist, height_hist))

    feature = 'regularity'
    group_data[feature].value_counts().plot(kind='bar', figsize=(width_hist, height_hist))

In [58]:
# find_bad_features_system('Tesla-V100', 50)

In [59]:
# find_bad_features_system('Tesla-A100', 50)

---
# bad performance when small cross row similarity for NVIDIA A100

In [ ]:
group_data = group_system_best[group_system_best['System']=='Tesla-A100']
group_data = group_data[group_data['mem_range'].isin(['[128-256]', '[256-512]'])]
group_data = group_data[(group_data['avg_nz_row']<=30) & (group_data['avg_nz_row']>=10)]
group_data = group_data[(group_data['avg_num_neighbours'] <= 0.05)]
print(np.round(sorted(set(group_data['avg_num_neighbours'])), 5))
print(group_data.shape[0], 'matrices',
      '// mean:', round(group_data['gflops'].mean()), 
      '// min:',  round(group_data['gflops'].min()), 
      '// max:',  round(group_data['gflops'].max()))

---
# "Ceteris paribus" analysis

In [ ]:
def plot_hued_features_sml(group_sys, system, hue_order_list, hue_order_list_specific, hue_order_list_specific_text, 
                           x_text, hue_text, xlabel, id_text, flag_filter, size_sml4=False):
    if(flag_filter):
        group_sys = group_sys.loc[group_sys[hue_text].isin(hue_order_list_specific)]
    else:
        hue_order_list_specific = hue_order_list
        hue_order_list_specific_text = hue_order_list

    fig, ax = plt.subplots()
    palette_choice = 'coolwarm'
    if(hue_text!='ranges_sml4'):
        sns.scatterplot(data=group_sys,y='gflops',palette=palette_choice, # marker='.', s=50,
            x=x_text,hue=hue_text,hue_order=hue_order_list_specific)

        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
        handles, labels = ax.get_legend_handles_labels()
        # ax.legend([i for i in handles], [i for i in hue_order_list_specific_text], loc='upper center', ncol =len(labels)//3)
        
        # ax.legend([(handles[0], handles[1], handles[2]), (handles[3], handles[4], handles[5])], 
        ax.legend([(handles[0], handles[1]), (handles[2], handles[3])], 
                  ['Small memory footprint, medium/large imbalance, small regularity', 
                   'Large memory footprint, balanced, medium/large regularity'], loc='upper left', 
                  handler_map={tuple: HandlerTuple(ndivide=None)}, bbox_to_anchor=(0, 1.34), fancybox=True, frameon=False)
        plt.xscale("log")
        ax.set_xticks([5, 10, 20, 50, 100, 500])
        ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
        
    else:
        if('avg_num_neighbours' in x_text):
            sns.scatterplot(data=group_sys,y='gflops',palette=palette_choice,
                # size=group_sys['cross_row_similarity'],
                x=x_text,hue=hue_text,hue_order=hue_order_list_specific)
            ax.set_xlim([0-.125,2+.125])
        else:
            sns.scatterplot(data=group_sys,y='gflops',palette=palette_choice,
                # size=group_sys['avg_num_neighbours'],
                x=x_text,hue=hue_text,hue_order=hue_order_list_specific)
            ax.set_xlim([0,1])

        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        handles, labels = ax.get_legend_handles_labels()
        # print(handles, labels)
        if(size_sml4):
            hue_order_list_specific_text_expanded = hue_order_list_specific_text + labels[len(hue_order_list_specific_text)+1:]
            # ax.legend([i for i in handles[1:]], [i for i in hue_order_list_specific_text_expanded], loc='upper left')#, ncol =len(labels)//2)
            
            # ax.legend([i for i in handles[1:]], [i for i in hue_order_list_specific_text_expanded], loc='upper left')#, ncol =len(labels)//2)
            ax.legend([(handles[1], handles[2]), (handles[3], handles[4])] + [i for i in handles[5:]], 
            ['Large memory footprint, small rows, medium/large imbalance', 
             'Small/Medium memory footprint, large rows, balanced'] + [i for i in labels[5:]], loc='upper left', 
            handler_map={tuple: HandlerTuple(ndivide=None)}, bbox_to_anchor=(0, 1.5), fancybox=True)
        else:
            # ax.legend([i for i in handles], [i for i in hue_order_list_specific_text], loc='upper left')

            # if we will group two markers of legend for "one text" to rule them all...
            # https://stackoverflow.com/questions/31478077/how-to-make-two-markers-share-the-same-label-in-the-legend-using-matplotlib
            ax.legend([(handles[0], handles[1]), (handles[2], handles[3])], 
                      ['Large memory footprint, small rows, medium/large imbalance', 
                       'Small/Medium memory footprint, large rows, balanced'], loc='upper left',
                      handler_map={tuple: HandlerTuple(ndivide=None)}, bbox_to_anchor=(0, 1.34), fancybox=True, frameon=False)

    plt.xlabel(xlabel)
    # plt.title(system + ' (all features fixed except one)')
    plt.ylabel('GFLOPs')

    plt.tight_layout()
    if(flag_filter):
        fig_name = './Plots/scatter_'+system+'_'+str(id_text)+'_filtered.pdf'
        plt.savefig(fig_name)
        plt.close()
        subprocess.run(['pdfcrop', fig_name], stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
        subprocess.run(['mv', fig_name.replace('.pdf', '-crop.pdf'), fig_name])
    else:
        fig_name = './Plots/scatter_'+system+'_'+str(id_text)+'.pdf'
        plt.savefig(fig_name)
        plt.close()
        subprocess.run(['pdfcrop', fig_name], stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
        subprocess.run(['mv', fig_name.replace('.pdf', '-crop.pdf'), fig_name])

#####################################################################

ranges_size2 = [ 'S',  'S',  'M',  'S',  'M',  'L',  'M',  'L',  'L']
# ranges_memr2 = ['[2-32]','[32-128]','[128-2048]'] # A_mem_footprint

#####################################################################
# these are good for NVIDIA-A100
# ranges_memr2 = ['[2-32]','[32-128]','[128-2048]'] # A_mem_footprint
# ranges_anr2  = ['[0-15]', '[15-150]', '[150-510]'] # avg_nz_row
# ranges_skew2 = ['[0-110]', '[110-1100]', '[1100-100000]'] # skew_coeff
# #####################################################################
# # these are good for AMD-EPYC-24
# ranges_memr2 = ['[2-32]','[32-128]','[128-2048]'] # A_mem_footprint
# ranges_anr2  = ['[0-15]', '[15-150]', '[150-510]'] # avg_nz_row
# ranges_skew2 = ['[0-110]', '[110-1100]', '[1100-100000]'] # skew_coeff
# #####################################################################

# avg nz/row, skew coeff, regularity
hue_order_list1 = ['SSS', 
                   'MSS', 'SMS', 'SSM', 
                   'LSS', 'SLS', 'SSL', 

                   'SMM', 'MSM', 'MMS', 
                   'MMM', 
                   'LMM', 'MLM', 'MML',
                   
                   'SML', 'SLM', 'MSL', 
                   'MLS', 'LMS', 'LSM',
                   
                   'SLL', 'LSL', 'LLS', 
                   'MLL', 'LML', 'LLM', 
                   'LLL']

# mem foot, skew coeff, regularity
hue_order_list2 = hue_order_list1

# mem foot, avg nz/row, regularity
hue_order_list3 = hue_order_list1

# mem foot, avg nz/row, skew coeff
hue_order_list4 = hue_order_list1

# ################################################################################
# # ranges_size3 = ['S', 'S', 'S', 'S', 'S', 'S', 'S', 
# #                 'M', 'M', 'M', 'M', 'M',  'M', 'M',  
# #                 'M', 'M', 'M', 'M', 'M', 'M',
# #                 'L', 'L', 'L', 'L', 'L', 'L', 'L']

# # group_system_best['ranges_sml1'] = group_system_best.apply (lambda row: set_category2(row['ranges_sml1'], hue_order_list1, ranges_size3, ranges_size_flag=True), axis=1)
# # group_system_best['ranges_sml2'] = group_system_best.apply (lambda row: set_category2(row['ranges_sml2'], hue_order_list2, ranges_size3, ranges_size_flag=True), axis=1)
# # group_system_best['ranges_sml3'] = group_system_best.apply (lambda row: set_category2(row['ranges_sml3'], hue_order_list3, ranges_size3, ranges_size_flag=True), axis=1)
# # group_system_best['ranges_sml4'] = group_system_best.apply (lambda row: set_category2(row['ranges_sml4'], hue_order_list4, ranges_size3, ranges_size_flag=True), axis=1)

# # hue_order_list1 = ['S', 'M', 'L']
# # hue_order_list2 = ['S', 'M', 'L']
# # hue_order_list3 = ['S', 'M', 'L']
# # hue_order_list4 = ['S', 'M', 'L']
# ################################################################################


In [ ]:
# ranges_dev = ['Tesla-A100', 'AMD-EPYC-24', 'Alveo-U280']
system = 'AMD-EPYC-24'

print('---', system, '---')
group_sys = group_system_best[group_system_best['System']==system]
print(group_sys.shape)

#####################################################################
# these are good for AMD-EPYC-24
ranges_memr2 = ['[2-32]','[32-128]','[128-2048]'] # A_mem_footprint
ranges_anr2  = ['[0-15]', '[15-150]', '[150-510]'] # avg_nz_row
ranges_skew2 = ['[0-110]', '[110-1100]', '[1100-100000]'] # skew_coeff
#####################################################################

group_sys['memr2']       = group_sys.apply (lambda row: set_category(row['A_mem_footprint'], ranges_memr2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['anr_categ2']  = group_sys.apply (lambda row: set_category(row['avg_nz_row'], ranges_anr2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['skew_categ2'] = group_sys.apply (lambda row: set_category(row['skew_coeff'], ranges_skew2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['regularity2'] = group_sys.apply (lambda row: set_category2(row['regularity'], ranges_regularity, ranges_size2, ranges_size_flag=True), axis=1)

group_sys['ranges_sml1'] = group_sys['anr_categ2'] + group_sys['skew_categ2'] + group_sys['regularity2']
group_sys['ranges_sml2'] = group_sys['memr2'] + group_sys['skew_categ2'] + group_sys['regularity2']
group_sys['ranges_sml3'] = group_sys['memr2'] + group_sys['anr_categ2'] + group_sys['regularity2']
group_sys['ranges_sml4'] = group_sys['memr2'] + group_sys['anr_categ2'] + group_sys['skew_categ2']

flag_filter = 1
# flag_filter = 0
hue_order_list_specific1 = ['SLS', 'SLM', 'SMS',   'LMM', 'LML', 'LSL']
hue_order_list_specific_text1 = [
    'SLS ( <15 nnz/row, >1000 skew, Small regularity )', 
    'SLM ( <15 nnz/row, >1000 skew, Medium regularity )', 
    'SMS ( <15 nnz/row, >100 skew, Small regularity )',
    'LMM ( >100 nnz/row, <1000 skew, Medium regularity )', 
    'LML ( >100 nnz/row, <1000 skew, Large regularity )', 
    'LSL ( >100 nnz/row, <10 skew, Large regularity )'
]

hue_order_list_specific2 = ['LLS', 'LLM', 'LMS',   'SSL', 'SMM', 'MSM']
hue_order_list_specific_text2 = [
    'LLS ( >128 MB, >1000 skew, Small regularity )', 
    'LLM ( >128 MB, >1000 skew, Medium regularity )', 
    'LMS ( >128 MB, >100 skew, Small regularity )',
    'SSL ( <32 MB, <10 skew, Large regularity )', 
    'SMM ( <32 MB, <1000 skew, Medium regularity )', 
    'MSM ( <128 MB, <10 skew, Medium regularity )'
]

hue_order_list_specific3 = ['LSS', 'LMS', 'LSM',   'SLM', 'SLL', 'MLM', 'MML', 'MLL']
hue_order_list_specific_text3 = [
    'LSS ( >128 MB, <15 nnz/row, Small regularity )', 
    'LMS ( >128 MB, <150 nnz/row, Small regularity )', 
    'LSM ( >128 MB, <15 nnz/row, Medium regularity )',
    'SLM ( <32 MB, >150 nnz/row, Medium regularity )', 
    'SLL ( <32 MB, >150 nnz/row, Large regularity )', 
    'MMM ( <128 MB, >15 nnz/row, Medium regularity )',
    'MLM ( <128 MB, >150 nnz/row, Medium regularity )',
    'MLL ( <128 MB, >150 nnz/row, Large regularity )'
]

# hue_order_list_specific4 = ['LSL', 'LSM', 'LML',       'SMM', 'SLM', 'MLS']
# hue_order_list_specific4 = ['LSM',   'MLS']
hue_order_list_specific4 = ['LSL', 
                            'LSM',

#                             'SMM', # ONE
#                             'MLS', # ONE

                            'SLS', # TWO
                            'MLS', # TWO

#                             'SMM', # THREE 
#                             'MMM', # THREE 

                           ]
hue_order_list_specific_text4 = [
    'LSL ( >128 MB, <15 nnz/row, >1000 skew )', 
    'LSM ( >128 MB, <15 nnz/row, >100 skew )', 

#     'SMM ( <32 MB, >15 nnz/row, <1000 skew )', # ONE
#     'MLS ( <128 MB, >150 nnz/row, <100 skew )', # ONE

    'SLS ( <32 MB, >150 nnz/row, <100 skew )', # TWO
    'MLS ( <128 MB, >150 nnz/row, <100 skew )', # TWO
    
#     'SMM ( <32 MB, >15 nnz/row, <1000 skew )', # THREE
#     'MMM ( <128 MB, >15 nnz/row, <1000 skew )', # THREE

    
    # 'LML ( >128 MB, <150 nnz/row, >1000 skew )', 
    # 'SLS ( <32 MB, >150 nnz/row, <100 skew )',
    # 'MLS ( <128 MB, >150 nnz/row, <100 skew )'
]

#####################################################################
plot_hued_features_sml(group_sys, system, hue_order_list1, hue_order_list_specific1, hue_order_list_specific_text1, 'A_mem_footprint', 'ranges_sml1', 'memory range (MB)', 1, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list2, hue_order_list_specific2, hue_order_list_specific_text2, 'avg_nz_row', 'ranges_sml2', 'average number of nonzeros per row', 2, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list3, hue_order_list_specific3, hue_order_list_specific_text3, 'skew_coeff', 'ranges_sml3', 'skew coefficient', 3, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list4, hue_order_list_specific4, hue_order_list_specific_text4, 'avg_num_neighbours', 'ranges_sml4', 'regularity ( avg_num_neighbours )', 4, flag_filter) #, size_sml4=True)
plot_hued_features_sml(group_sys, system, hue_order_list4, hue_order_list_specific4, hue_order_list_specific_text4, 'cross_row_similarity', 'ranges_sml4', 'regularity ( cross_row_similarity )', 5, flag_filter)    
#####################################################################
rc = subprocess.call('cd ./Plots; bash bash_it.sh '+system+'; pdfcrop scatter_'+system+'_4_filtered.pdf; cd ../', shell=True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)


In [ ]:
system = 'Tesla-A100'

print('---', system, '---')
group_sys = group_system_best[group_system_best['System']==system]
print(group_sys.shape)

#####################################################################
# these are good for Tesla-A100
ranges_memr2 = ['[2-32]','[32-128]','[128-2048]'] # A_mem_footprint
ranges_anr2  = ['[0-15]', '[15-150]', '[150-510]'] # avg_nz_row
ranges_skew2 = ['[0-110]', '[110-1100]', '[1100-100000]'] # skew_coeff
#####################################################################

group_sys['memr2']       = group_sys.apply (lambda row: set_category(row['A_mem_footprint'], ranges_memr2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['anr_categ2']  = group_sys.apply (lambda row: set_category(row['avg_nz_row'], ranges_anr2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['skew_categ2'] = group_sys.apply (lambda row: set_category(row['skew_coeff'], ranges_skew2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['regularity2'] = group_sys.apply (lambda row: set_category2(row['regularity'], ranges_regularity, ranges_size2, ranges_size_flag=True), axis=1)

group_sys['ranges_sml1'] = group_sys['anr_categ2'] + group_sys['skew_categ2'] + group_sys['regularity2']
group_sys['ranges_sml2'] = group_sys['memr2'] + group_sys['skew_categ2'] + group_sys['regularity2']
group_sys['ranges_sml3'] = group_sys['memr2'] + group_sys['anr_categ2'] + group_sys['regularity2']
group_sys['ranges_sml4'] = group_sys['memr2'] + group_sys['anr_categ2'] + group_sys['skew_categ2']

flag_filter = 1
hue_order_list_specific1 = ['SLS', 
                            # 'SLM', 
                            'SMS',   
                            # 'LMM', 
                            'LML', 
                            'LSL']
hue_order_list_specific_text1 = [
    'SLS ( <15 nnz/row, >1000 skew, Small regularity )', 
    # 'SLM ( <15 nnz/row, >1000 skew, Medium regularity )', 
    'SMS ( <15 nnz/row, >100 skew, Small regularity )',
    # 'LMM ( >100 nnz/row, <1000 skew, Medium regularity )', 
    'LML ( >100 nnz/row, <1000 skew, Large regularity )', 
    'LSL ( >100 nnz/row, <10 skew, Large regularity )'
]

hue_order_list_specific2 = [
    'SLS',
    'SMS',
#     'MLS', 
#     'LSM',

    'LSL', # ONE
    'LML',# ONE
#     'MSL', # TWO
#     'LSL', # TWO
]
hue_order_list_specific_text2 = [
    'SLS ( <32 MB, >1000 skew, Small regularity )',
    'SMS ( <32 MB, >100 skew, Small regularity )',
#     'MLS ( <128 MB, >1000 skew, Small regularity )',
#     'LSM ( >128 MB, <100 skew, Medium regularity )',

    'LSL ( >128 MB, <100 skew, Large regularity )', # ONE
    'LML ( >128 MB, <1000 skew, Large regularity )', # ONE
#     'MSL ( >32 MB, <100 skew, Large regularity )', # TWO
#     'LSL ( >128 MB, <100 skew, Large regularity )', # TWO
]

hue_order_list_specific3 = ['SSS', 'SMS', 'SSM',    'LML', 'LLM', 'LLL']
hue_order_list_specific_text3 = [
    'SSS ( <32 MB, <15 nnz/row, Small regularity )', 
    'SMS ( <32 MB, <150 nnz/row, Small regularity )', 
    'SSM ( <32 MB, <15 nnz/row, Medium regularity )',

    'LML ( >128 MB, >15 nnz/row, Large regularity )', 
    'LLM ( >128 MB, >150 nnz/row, Medium regularity )', 
    'LLL ( >128 MB, >150 nnz/row, Large regularity )', 
]

# hue_order_list_specific4 = ['SSL', 'SML', 'SSM',   'LMM', 'LLM', 'LLS']
hue_order_list_specific4 = ['SML', 'SSM',   'LLM','LLS']
hue_order_list_specific_text4 = [
    # 'SSL ( <32 MB, <15 nnz/row, >1000 skew )', 
    'SML ( <32 MB, <150 nnz/row, >1000 skew )', 
    'SSM ( <32 MB, <15 nnz/row, >100 skew )', 
    # 'LMM ( >128 MB, >15 nnz/row, <1000 skew )',
    'LLM ( >128 MB, >150 nnz/row, <1000 skew )',
    'LLS ( >128 MB, >150 nnz/row, <10 skew )'
]

#####################################################################
plot_hued_features_sml(group_sys, system, hue_order_list1, hue_order_list_specific1, hue_order_list_specific_text1, 'A_mem_footprint', 'ranges_sml1', 'memory range (MB)', 1, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list2, hue_order_list_specific2, hue_order_list_specific_text2, 'avg_nz_row', 'ranges_sml2', 'average number of nonzeros per row', 2, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list3, hue_order_list_specific3, hue_order_list_specific_text3, 'skew_coeff', 'ranges_sml3', 'skew coefficient', 3, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list4, hue_order_list_specific4, hue_order_list_specific_text4, 'avg_num_neighbours', 'ranges_sml4', 'regularity ( avg_num_neighbours )', 4, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list4, hue_order_list_specific4, hue_order_list_specific_text4, 'cross_row_similarity', 'ranges_sml4', 'regularity ( cross_row_similarity )', 5, flag_filter)    
#####################################################################
rc = subprocess.call('cd ./Plots; bash bash_it.sh '+system+'; pdfcrop scatter_'+system+'_2_filtered.pdf; cd ../', shell=True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)


In [ ]:
# ranges_dev = ['Tesla-A100', 'AMD-EPYC-24', 'Alveo-U280']
system = 'Alveo-U280'

print('---', system, '---')
group_sys = group_system_best[group_system_best['System']==system]
print(group_sys.shape)

#####################################################################
# these are good for AMD-EPYC-24
ranges_memr2 = ['[2-32]','[32-128]','[128-2048]'] # A_mem_footprint
ranges_anr2  = ['[0-15]', '[15-150]', '[150-510]'] # avg_nz_row
ranges_skew2 = ['[0-110]', '[110-1100]', '[1100-100000]'] # skew_coeff
#####################################################################

group_sys['memr2']       = group_sys.apply (lambda row: set_category(row['A_mem_footprint'], ranges_memr2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['anr_categ2']  = group_sys.apply (lambda row: set_category(row['avg_nz_row'], ranges_anr2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['skew_categ2'] = group_sys.apply (lambda row: set_category(row['skew_coeff'], ranges_skew2, ranges_size, ranges_size_flag=True), axis=1)
group_sys['regularity2'] = group_sys.apply (lambda row: set_category2(row['regularity'], ranges_regularity, ranges_size2, ranges_size_flag=True), axis=1)

group_sys['ranges_sml1'] = group_sys['anr_categ2'] + group_sys['skew_categ2'] + group_sys['regularity2']
group_sys['ranges_sml2'] = group_sys['memr2'] + group_sys['skew_categ2'] + group_sys['regularity2']
group_sys['ranges_sml3'] = group_sys['memr2'] + group_sys['anr_categ2'] + group_sys['regularity2']
group_sys['ranges_sml4'] = group_sys['memr2'] + group_sys['anr_categ2'] + group_sys['skew_categ2']

flag_filter = 1
# flag_filter = 0
hue_order_list_specific1 = ['SLS', 'SLM', 'SMS',   'LMM', 'LML', 'LSL']
hue_order_list_specific_text1 = [
    'SLS ( <15 nnz/row, >1000 skew, Small regularity )', 
    'SLM ( <15 nnz/row, >1000 skew, Medium regularity )', 
    'SMS ( <15 nnz/row, >100 skew, Small regularity )',
    'LMM ( >100 nnz/row, <1000 skew, Medium regularity )', 
    'LML ( >100 nnz/row, <1000 skew, Large regularity )', 
    'LSL ( >100 nnz/row, <10 skew, Large regularity )'
]

hue_order_list_specific2 = ['LLS', 'LLM', 'LMS',   'SSL', 'SMM', 'MSM']
hue_order_list_specific_text2 = [
    'LLS ( >128 MB, >1000 skew, Small regularity )', 
    'LLM ( >128 MB, >1000 skew, Medium regularity )', 
    'LMS ( >128 MB, >100 skew, Small regularity )',
    'SSL ( <32 MB, <10 skew, Large regularity )', 
    'SMM ( <32 MB, <1000 skew, Medium regularity )', 
    'MSM ( <128 MB, <10 skew, Medium regularity )'
]

hue_order_list_specific3 = ['LSS', 'LMS', 'LSM',   'SLM', 'SLL', 'MLM', 'MML', 'MLL']
hue_order_list_specific_text3 = [
    'LSS ( >128 MB, <15 nnz/row, Small regularity )', 
    'LMS ( >128 MB, <150 nnz/row, Small regularity )', 
    'LSM ( >128 MB, <15 nnz/row, Medium regularity )',
    'SLM ( <32 MB, >150 nnz/row, Medium regularity )', 
    'SLL ( <32 MB, >150 nnz/row, Large regularity )', 
    'MMM ( <128 MB, >15 nnz/row, Medium regularity )',
    'MLM ( <128 MB, >150 nnz/row, Medium regularity )',
    'MLL ( <128 MB, >150 nnz/row, Large regularity )'
]

# hue_order_list_specific4 = ['LSL', 'LSM', 'LML',       'SMM', 'SLM', 'MLS']
# hue_order_list_specific4 = ['LSM',   'MLS']
hue_order_list_specific4 = ['LSL', 
                            'LSM',

#                             'SMM', # ONE
#                             'MLS', # ONE

                            'SLS', # TWO
                            'MLS', # TWO

#                             'SMM', # THREE 
#                             'MMM', # THREE 

                           ]
hue_order_list_specific_text4 = [
    'LSL ( >128 MB, <15 nnz/row, >1000 skew )', 
    'LSM ( >128 MB, <15 nnz/row, >100 skew )', 

#     'SMM ( <32 MB, >15 nnz/row, <1000 skew )', # ONE
#     'MLS ( <128 MB, >150 nnz/row, <100 skew )', # ONE

    'SLS ( <32 MB, >150 nnz/row, <100 skew )', # TWO
    'MLS ( <128 MB, >150 nnz/row, <100 skew )', # TWO
    
#     'SMM ( <32 MB, >15 nnz/row, <1000 skew )', # THREE
#     'MMM ( <128 MB, >15 nnz/row, <1000 skew )', # THREE

    
    # 'LML ( >128 MB, <150 nnz/row, >1000 skew )', 
    # 'SLS ( <32 MB, >150 nnz/row, <100 skew )',
    # 'MLS ( <128 MB, >150 nnz/row, <100 skew )'
]

#####################################################################
plot_hued_features_sml(group_sys, system, hue_order_list1, hue_order_list_specific1, hue_order_list_specific_text1, 'A_mem_footprint', 'ranges_sml1', 'memory range (MB)', 1, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list2, hue_order_list_specific2, hue_order_list_specific_text2, 'avg_nz_row', 'ranges_sml2', 'average number of nonzeros per row', 2, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list3, hue_order_list_specific3, hue_order_list_specific_text3, 'skew_coeff', 'ranges_sml3', 'skew coefficient', 3, flag_filter)
plot_hued_features_sml(group_sys, system, hue_order_list4, hue_order_list_specific4, hue_order_list_specific_text4, 'avg_num_neighbours', 'ranges_sml4', 'regularity ( avg_num_neighbours )', 4, flag_filter) #, size_sml4=True)
plot_hued_features_sml(group_sys, system, hue_order_list4, hue_order_list_specific4, hue_order_list_specific_text4, 'cross_row_similarity', 'ranges_sml4', 'regularity ( cross_row_similarity )', 5, flag_filter)    
#####################################################################
rc = subprocess.call('cd ./Plots; bash bash_it.sh '+system+'; pdfcrop scatter_'+system+'_4_filtered.pdf; cd ../', shell=True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)


---

---
# GL7D19_friends

In [ ]:
def read_gpu_csv(filename):
    header_names = ['mtx_name','distribution','placement','seed',
                    'm','n','nz','density','A_mem_footprint','mem_range',
                    'avg_nz_row','std_nz_row',
                    'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                    'avg_scattering','std_scattering','avg_scattering_scaled','std_scattering_scaled',
                    'skew_coeff','avg_num_neighbours','cross_row_similarity',
                    'implementation','time','gflops','W_avg','J_estimated', 'System', 'Arch']

    df = pd.read_csv(filename, names = header_names)
    df['System'] = 'Tesla-A100'
    df['Arch'] = 'GPU'

    # Keep mean of benchmarks, store in "groupreps", take mean
    header_group_by = [x for x in header_names if x not in  ['time','gflops','W_avg','J_estimated']]
    df_groupreps = df.groupby(header_group_by, observed = True).mean().reset_index().reindex(columns=header_names)
    # print(df_groupreps.shape)

    # Group per system, take best (it was over "inputdata", but "groupreps" is better choice I think)
    # fixed this after reordering groupreps columns according to header_names
    # before this, columns and data were mixed and it was a complete shitstorm
    header_group_by = [x for x in header_names if x not in  ['implementation', 'time','gflops','W_avg','J_estimated']]
    df_group_system = df_groupreps.groupby(header_group_by, as_index = False, observed = True)
    reslist = []
    for desc, experiment in df_group_system:
        best_format = experiment['implementation'].iloc[experiment['gflops'].argmax()]
        outrow = experiment[experiment['implementation'] == best_format]
        # if(len(outrow)>1):
        #     print(len(outrow), outrow)
        reslist.append(outrow.values.tolist()[0])             

    df_group_system_best = pd.DataFrame(reslist, columns = header_names)
    # print(df_group_system_best.shape)
    return df_groupreps, df_group_system_best


---
# test large vs small regularity (and small vs large)

In [ ]:
%%time
# small_large_regularity_groupreps, small_large_regularity_group_system_best = read_gpu_csv('../benchmark_results/epyc5-A100/gl_friend_small_large_regularity.csv')
# large_small_regularity_groupreps, large_small_regularity_group_system_best = read_gpu_csv('../benchmark_results/epyc5-A100/gl_friend_large_small_regularity.csv')
small_large_regularity_groupreps, small_large_regularity_group_system_best = read_gpu_csv('../benchmark_results/epyc5-A100/small_large_regularity.csv')
large_small_regularity_groupreps, large_small_regularity_group_system_best = read_gpu_csv('../benchmark_results/epyc5-A100/large_small_regularity.csv')


In [ ]:
small_large_regularity_groupreps.sort_values(by=['A_mem_footprint', 'implementation', 'cross_row_similarity'], inplace=True, ignore_index=True)
display(small_large_regularity_groupreps[['A_mem_footprint', 'implementation','avg_num_neighbours', 'cross_row_similarity', 'gflops']])

In [ ]:
large_small_regularity_groupreps.sort_values(by=['A_mem_footprint', 'implementation', 'avg_num_neighbours'], inplace=True, ignore_index=True)
display(large_small_regularity_groupreps[['A_mem_footprint', 'implementation','avg_num_neighbours', 'cross_row_similarity', 'gflops']])

---
# GL Friends

In [ ]:
%%time
GL_friends_groupreps, GL_friends_group_system_best = read_gpu_csv('../benchmark_results/epyc5-A100/GL_friends.csv')
GL_friends_10x_reg_groupreps, GL_friends_10x_reg_group_system_best = read_gpu_csv('../benchmark_results/epyc5-A100/GL_friends_10x_reg.csv')
GL_friends_100x_reg_groupreps, GL_friends_100x_reg_group_system_best = read_gpu_csv('../benchmark_results/epyc5-A100/GL_friends_100x_reg.csv')


In [ ]:
plt.rc('figure', figsize=(2*width,2*height))
    
ranges_dev = ['Tesla-A100']
for System in ranges_dev:
    plot_histogram_performance_device(GL_friends_100x_reg_group_system_best, 'bestof', System)
    plot_histogram_performance_device(GL_friends_100x_reg_groupreps, 'all', System)

In [ ]:
def print_df_stats(df):
    print(round(df['gflops'].min()), '-', round(df['gflops'].max()), '( mean =', round(df['gflops'].mean()), ')')
    
print_df_stats(GL_friends_group_system_best)
print_df_stats(GL_friends_10x_reg_group_system_best)
print_df_stats(GL_friends_100x_reg_group_system_best)


In [ ]:
GL_friends_group_system_best.columns

In [ ]:
def print_opt_perf_pcg(df):
    df['optimal_performance'] = 2*df['nz'] / ((8*df['n'] + 12*df['m'] + 12*df['nz'] + 4)/1350)
    df['opt_perf_pcg'] = np.round(df['gflops'] / df['optimal_performance'] * 100,2)

    pd.set_option('display.max_rows', len(df))
    display(df[['optimal_performance', 'gflops', 'opt_perf_pcg']]) 
    pd.reset_option('display.max_rows')
    
print_opt_perf_pcg(GL_friends_group_system_best)
print_opt_perf_pcg(GL_friends_10x_reg_group_system_best)
print_opt_perf_pcg(GL_friends_100x_reg_group_system_best)


---
# New entries